# Training of Selected Model:

In [2]:
import numpy as np
import tensorflow as tf
import time
import datetime
from scipy import stats
import math
import os
import itertools

import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats.distributions import norm
from sklearn.model_selection import train_test_split
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D

In [3]:
os.chdir('/Users/tugce/Desktop/Results')

In [4]:
#Data load:
model = 'GBM'
X_train=np.loadtxt('X_train_%s'% model)
y_train=np.loadtxt('y_train_%s'% model)
y_train=y_train.reshape(len(y_train),1)
X_test=np.loadtxt('X_test_%s'% model)
y_test=np.loadtxt('y_test_%s'% model)
y_test=y_test.reshape(len(y_test),1)
X_val=np.loadtxt('X_val_%s'% model)
y_val=np.loadtxt('y_val_%s'% model)
y_val=y_val.reshape(len(y_val),1)
X_data_H=np.loadtxt('X_data_H_%s'% model)
y_data_H=np.loadtxt('y_data_H_%s'% model)
y_data_H=y_data_H.reshape(len(y_data_H),1)

In [5]:
#Activation functions:
def buildLayer(a, Type):  
    if Type == 'leaky':
        layer = tf.nn.leaky_relu(a, name=None)
    elif Type =='relu':
        layer =tf.nn.relu(a,name=None)
    elif Type == 'elu':
        layer = tf.nn.elu(a, name=None)
    elif Type == 'sig':
        layer = tf.nn.sigmoid(a, name = None)
    elif Type == 'tanh':
        layer = tf.nn.tanh(a, name = None)
    else:
        print('Wrong type of layers!')
    return layer

In [6]:
def buildNN(numLayers, typeLayers, numNeurons, learning_rate, epoch, batch_size, display_step, dropout_rate, X_train, y_train, X_test, y_test, X_val, y_val, X_data_H, y_data_H,Layer_index=1,model='GBM'):
    #typeLayers is a list of type of layers, numNeurons is a list of # of neurons in each layer
     
    tf.reset_default_graph()
    start_time = time.time()
    iterations = int(X_train.shape[0]/batch_size)
    X_index = np.arange(X_train.shape[0])
    n_input = X_train.shape[1]
    n_classes = 1
    X = tf.placeholder('float', [None, n_input],name='X')
    Y = tf.placeholder('float', [None, n_classes],name='Y')
    drop_rate = tf.placeholder('float',name='drop_rate')
     
    # Store layers weight & bias
    h = []
    b = []
    weightslist = [tf.Variable(tf.random_normal([n_input, numNeurons[0]], stddev= 1/(X_train.shape[0])))]
    biaslist = []
    for i in range(numLayers):
        h.append('h' + str(i + 1))
        b.append('b' + str(i + 1))
        biaslist.append(tf.Variable(tf.random_normal([numNeurons[i]], stddev= 1/(X_train.shape[0]))))
        if i > 0:
            weightslist.append(tf.Variable(tf.random_normal([numNeurons[i - 1], numNeurons[i]], stddev= 1/(X_train.shape[0]))))
    b.append('out')
    h.append('out')
    weightslist.append(tf.Variable(tf.random_normal([numNeurons[numLayers - 1], n_classes], stddev= 1/(X_train.shape[0]))))                          
    biaslist.append(tf.Variable(tf.random_normal([n_classes], stddev= 1/(X_train.shape[0]))))                         
    hcom = zip(h, weightslist)
    weights = dict(hcom)
    bcom = zip(b, biaslist)
    biases = dict(bcom)
     
    a = tf.add(tf.matmul(X, weights['h1']), biases['b1'])
    layer = buildLayer(a, typeLayers[0])
    drop = tf.nn.dropout(layer, keep_prob=1-drop_rate)
    for i in range(1,numLayers):
        a = tf.add(tf.matmul(drop, weights[h[i]]), biases[b[i]])
        layer = buildLayer(a, typeLayers[i])
        drop = tf.nn.dropout(layer, keep_prob=1-drop_rate)
    output_a = tf.add(tf.matmul(drop, weights['out']), biases['out'])  
    logits = tf.exp(output_a, name='logits')
    
    loss = tf.reduce_mean(tf.square(tf.subtract(logits, Y)))
     
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    #optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss)
   
    #Saving the trained model:    
    saver = tf.train.Saver()
    
    init = tf.global_variables_initializer()
   
    with tf.Session() as sess:
        sess.run(init) 
        print("Train on {} samples,".format(len(y_train)),"validate on {} samples,".format(len(y_val)),"test on {} samples".format(len(y_test)))
        for epc in range(epoch):
            np.random.shuffle(X_index)
            avg_loss = 0.0
            for itr in range(iterations):
                sub_index = X_index[itr * batch_size: (1 + itr) * batch_size - 1]
                x_batch = X_train[sub_index]
                y_batch = y_train[sub_index]
                _, loss_val = sess.run([train_op,loss], feed_dict= {X:x_batch, Y:y_batch, drop_rate: dropout_rate})
                avg_loss += loss_val/iterations
            sub_val_loss = sess.run(loss, feed_dict={X:X_val, Y:y_val, drop_rate:0})
            print("Epoch:{:2d}".format(epc+1), "average loss:", "{0:.8f}".format(avg_loss), "total validation loss:", "{0:.8f}".format(sub_val_loss))
        
        #Save the final model:
        meta = './model_%s_' %model+'{}layers_{}neurons_result_{}'.format(numLayers,numNeurons[0],Layer_index)
        #saver.save(sess,meta,global_step=1000)
        training_time = (((time.time() - start_time))/60)
        saver.save(sess,meta)
        
        
        print("Training Time : %s minutes " % training_time)  
        test_loss = sess.run([loss], feed_dict = {X:X_test, Y:y_test, drop_rate:0})
        print("total test loss of {}: {}".format(typeLayers,test_loss))
        train_loss = sess.run([loss],feed_dict = {X:X_data_H, Y:y_data_H, drop_rate:0})
        print("total train loss of {}: {}".format(typeLayers,train_loss))
        y_train_pred = sess.run([logits],feed_dict={X:X_data_H, Y:y_data_H, drop_rate:0})
        start=time.time()
        y_test_pred = sess.run([logits],feed_dict={X:X_test, Y:y_test, drop_rate:0})
        print(y_test_pred[0:5])
        predict_time = (((time.time() - start)*1000000/len(y_test)))
        print("Average prediction time: {}".format(predict_time))
        W, B = sess.run([weights, biases])
     

    return training_time

In [17]:
#Testing against original paper:
buildNN(4, ['leaky','elu','relu','elu'], [120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test, y_test, X_val, y_val, X_data_H, y_data_H,1,'GBM')

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00771474 total validation loss: 0.00020515
Epoch: 2 average loss: 0.00076058 total validation loss: 0.00012173
Epoch: 3 average loss: 0.00063036 total validation loss: 0.00008639
Epoch: 4 average loss: 0.00047230 total validation loss: 0.00006364
Epoch: 5 average loss: 0.00037367 total validation loss: 0.00014439
Epoch: 6 average loss: 0.00032600 total validation loss: 0.00005076
Epoch: 7 average loss: 0.00030832 total validation loss: 0.00009682
Epoch: 8 average loss: 0.00029058 total validation loss: 0.00006376
Epoch: 9 average loss: 0.00028238 total validation loss: 0.00004651
Epoch:10 average loss: 0.00026540 total validation loss: 0.00005517
Training Time : 1.3021855354309082 minutes 
total test loss of ['leaky', 'elu', 'relu', 'elu']: [5.548529e-05]
total train loss of ['leaky', 'elu', 'relu', 'elu']: [5.5579694e-05]
[array([[0.34463152],
       [0.4935505 ],
       [0.49099463],
  

1.3021855354309082

In [15]:
def training(num_Layers,num_Neurons,model,learning_rate = 0.001,epoch =10,batch_size = 64,display_step=1,dropout_rate=0.25):
    #Data loading:
    X_train=np.loadtxt('X_train_%s'% model)
    y_train=np.loadtxt('y_train_%s'% model)
    y_train=y_train.reshape(len(y_train),1)
    X_test=np.loadtxt('X_test_%s'% model)
    y_test=np.loadtxt('y_test_%s'% model)
    y_test=y_test.reshape(len(y_test),1)
    X_val=np.loadtxt('X_val_%s'% model)
    y_val=np.loadtxt('y_val_%s'% model)
    y_val=y_val.reshape(len(y_val),1)
    X_data_H=np.loadtxt('X_data_H_%s'% model)
    y_data_H=np.loadtxt('y_data_H_%s'% model)
    y_data_H=y_data_H.reshape(len(y_data_H),1)
    
    #Generation of different activation function series:
    set_of_activations = ['elu','leaky','relu']
    set_of_layers = [j for j in set(list(itertools.product(set_of_activations,repeat = num_Layers-1)))]
    set_of_layers = sorted(set_of_layers)
    #set_of_layers = [['elu','elu','elu','elu','elu','elu'],['leaky','leaky','leaky','leaky','leaky','leaky'],['relu','relu','relu','relu','relu','relu']]
    training_time = []
    #Number of neurons set:
    Neurons=np.ones((num_Layers,), dtype=int)*num_Neurons
    for i in range(len(set_of_layers)):
        layers = list(['leaky']+list(set_of_layers[i]))
        print("**********************Training {} - {}:".format(i+1,layers))
        train_time = buildNN(num_Layers, layers, Neurons, learning_rate, epoch, batch_size, display_step, dropout_rate, X_train, y_train, X_test, y_test, X_val, y_val, X_data_H, y_data_H,(i+1),model)
        training_time.append(train_time) 
    np.savetxt('training_time_%s'% model+'{}layers_{}neurons'.format(num_Layers,num_Neurons),training_time)
    return set_of_layers,training_time
    

In [8]:
#First run -- 2 layers & 'GBM' :
for j in np.arange(20,140,20):
    print("%%%%%%%%%%%%%%%%% Training 2 Layers & {} Neurons:".format(j))
    set_of_layers,training_time = training(2,j,'GBM')

%%%%%%%%%%%%%%%%% Training 2 Layers & 20 Neurons:
**********************Training 1 - ('elu', 'elu'):
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.02030234 total validation loss: 0.00208591
Epoch: 2 average loss: 0.00356989 total validation loss: 0.00199279
Epoch: 3 average loss: 0.00267925 total validation loss: 0.00152227
Epoch: 4 average loss: 0.00194909 total validation loss: 0.00128456
Epoch: 5 average loss: 0.00171109 total validation loss: 0.00120291
Epoch: 6 average loss: 0.00165232 total validation loss: 0.00118293
Epoch: 7 average loss: 0.00163224 total validation loss: 0.00136463
Epoch: 8 average loss: 0.00160786 total validation loss: 0.00111287
Epoch: 9 average loss: 0.00156690 total validation loss: 0.00105715
Epoch:10 average loss: 0.00153773 total validation loss: 0.00098503
Training Time : 0.481783131758372 minutes 
total test loss of ('elu', 'elu'): [0.0009615274]
total train loss of ('elu', 'elu'): [0.00096658414]


Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.02879640 total validation loss: 0.00291459
Epoch: 2 average loss: 0.00408809 total validation loss: 0.00133078
Epoch: 3 average loss: 0.00312154 total validation loss: 0.00088091
Epoch: 4 average loss: 0.00241244 total validation loss: 0.00071570
Epoch: 5 average loss: 0.00220156 total validation loss: 0.00077002
Epoch: 6 average loss: 0.00203524 total validation loss: 0.00069326
Epoch: 7 average loss: 0.00186191 total validation loss: 0.00075599
Epoch: 8 average loss: 0.00178664 total validation loss: 0.00090435
Epoch: 9 average loss: 0.00173229 total validation loss: 0.00090384
Epoch:10 average loss: 0.00170492 total validation loss: 0.00084007
Training Time : 0.5024634401003519 minutes 
total test loss of ('relu', 'leaky'): [0.0008373396]
total train loss of ('relu', 'leaky'): [0.0008313544]
[array([[0.33871472],
       [0.47925133],
       [0.47925133],
       ...,
       [0.34637812]

Epoch: 1 average loss: 0.01624146 total validation loss: 0.00061902
Epoch: 2 average loss: 0.00152520 total validation loss: 0.00033643
Epoch: 3 average loss: 0.00094217 total validation loss: 0.00026483
Epoch: 4 average loss: 0.00082538 total validation loss: 0.00019776
Epoch: 5 average loss: 0.00079244 total validation loss: 0.00017932
Epoch: 6 average loss: 0.00077254 total validation loss: 0.00017126
Epoch: 7 average loss: 0.00076628 total validation loss: 0.00015219
Epoch: 8 average loss: 0.00075308 total validation loss: 0.00016035
Epoch: 9 average loss: 0.00074974 total validation loss: 0.00021727
Epoch:10 average loss: 0.00074281 total validation loss: 0.00013854
Training Time : 0.591840120156606 minutes 
total test loss of ('leaky', 'relu'): [0.00014200101]
total train loss of ('leaky', 'relu'): [0.00013838285]
[array([[0.3402789 ],
       [0.4978002 ],
       [0.48903474],
       ...,
       [0.36211514],
       [0.13891299],
       [0.10537928]], dtype=float32)]
Average pred

Epoch: 2 average loss: 0.00102900 total validation loss: 0.00027474
Epoch: 3 average loss: 0.00084381 total validation loss: 0.00029576
Epoch: 4 average loss: 0.00069235 total validation loss: 0.00030989
Epoch: 5 average loss: 0.00063332 total validation loss: 0.00030693
Epoch: 6 average loss: 0.00061777 total validation loss: 0.00029382
Epoch: 7 average loss: 0.00060737 total validation loss: 0.00024462
Epoch: 8 average loss: 0.00060240 total validation loss: 0.00029294
Epoch: 9 average loss: 0.00059388 total validation loss: 0.00029681
Epoch:10 average loss: 0.00058455 total validation loss: 0.00030690
Training Time : 0.6393918514251709 minutes 
total test loss of ('leaky', 'elu'): [0.00029523848]
total train loss of ('leaky', 'elu'): [0.0002980468]
[array([[0.33502066],
       [0.47446504],
       [0.45861816],
       ...,
       [0.3524441 ],
       [0.12749451],
       [0.10006337]], dtype=float32)]
Average prediction time: 1.3494332631429036
**********************Training 5 - ('l

Epoch: 3 average loss: 0.00101784 total validation loss: 0.00036746
Epoch: 4 average loss: 0.00072135 total validation loss: 0.00031782
Epoch: 5 average loss: 0.00060846 total validation loss: 0.00029596
Epoch: 6 average loss: 0.00057798 total validation loss: 0.00027244
Epoch: 7 average loss: 0.00057119 total validation loss: 0.00023713
Epoch: 8 average loss: 0.00056210 total validation loss: 0.00030162
Epoch: 9 average loss: 0.00055391 total validation loss: 0.00034280
Epoch:10 average loss: 0.00054505 total validation loss: 0.00026142
Training Time : 0.7167242368062338 minutes 
total test loss of ('elu', 'leaky'): [0.00025321223]
total train loss of ('elu', 'leaky'): [0.00025626703]
[array([[0.35057247],
       [0.4751539 ],
       [0.47407785],
       ...,
       [0.36909765],
       [0.13172017],
       [0.10582428]], dtype=float32)]
Average prediction time: 2.039353052775065
**********************Training 3 - ('elu', 'relu'):
Train on 216000 samples, validate on 24000 samples, te

Epoch: 5 average loss: 0.00083361 total validation loss: 0.00026307
Epoch: 6 average loss: 0.00083463 total validation loss: 0.00021645
Epoch: 7 average loss: 0.00083382 total validation loss: 0.00026694
Epoch: 8 average loss: 0.00083104 total validation loss: 0.00023569
Epoch: 9 average loss: 0.00083177 total validation loss: 0.00021192
Epoch:10 average loss: 0.00082922 total validation loss: 0.00021822
Training Time : 0.5725068052609762 minutes 
total test loss of ('relu', 'relu'): [0.00022177552]
total train loss of ('relu', 'relu'): [0.00021842802]
[array([[0.33867425],
       [0.48505613],
       [0.49151498],
       ...,
       [0.3663572 ],
       [0.12774697],
       [0.09770826]], dtype=float32)]
Average prediction time: 2.0760297775268555
%%%%%%%%%%%%%%%%% Training 2 Layers & 100 Neurons:
**********************Training 1 - ('elu', 'elu'):
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00971334 total validation loss: 0.001879

Epoch: 6 average loss: 0.00058287 total validation loss: 0.00021643
Epoch: 7 average loss: 0.00056434 total validation loss: 0.00014110
Epoch: 8 average loss: 0.00055263 total validation loss: 0.00017228
Epoch: 9 average loss: 0.00053620 total validation loss: 0.00017048
Epoch:10 average loss: 0.00051665 total validation loss: 0.00016359
Training Time : 0.6423448999722798 minutes 
total test loss of ('relu', 'elu'): [0.0001644192]
total train loss of ('relu', 'elu'): [0.00016307074]
[array([[0.33854616],
       [0.48378712],
       [0.4809566 ],
       ...,
       [0.36184162],
       [0.1398947 ],
       [0.10505259]], dtype=float32)]
Average prediction time: 2.459820111592611
**********************Training 8 - ('relu', 'leaky'):
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01102379 total validation loss: 0.00031932
Epoch: 2 average loss: 0.00088372 total validation loss: 0.00026262
Epoch: 3 average loss: 0.00074357 total validatio

Epoch: 7 average loss: 0.00025214 total validation loss: 0.00008796
Epoch: 8 average loss: 0.00024643 total validation loss: 0.00008434
Epoch: 9 average loss: 0.00023559 total validation loss: 0.00007221
Epoch:10 average loss: 0.00022762 total validation loss: 0.00009599
Training Time : 0.7342254837354024 minutes 
total test loss of ('leaky', 'leaky'): [9.196886e-05]
total train loss of ('leaky', 'leaky'): [9.296831e-05]
[array([[0.34018672],
       [0.46895245],
       [0.4703865 ],
       ...,
       [0.3670138 ],
       [0.14129986],
       [0.10240321]], dtype=float32)]
Average prediction time: 3.6849339803059897
**********************Training 6 - ('leaky', 'relu'):
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01054969 total validation loss: 0.00021480
Epoch: 2 average loss: 0.00086406 total validation loss: 0.00013727
Epoch: 3 average loss: 0.00053514 total validation loss: 0.00009612
Epoch: 4 average loss: 0.00038892 total val

In [29]:
#Second run -- 5 layers & 120 neurons & 'GBM' :
set_of_layers,training_time = training(5,120,'GBM')

**********************Training 1 - ['leaky', 'relu', 'elu', 'elu', 'elu']:
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00624431 total validation loss: 0.00016788
Epoch: 2 average loss: 0.00075453 total validation loss: 0.00009336
Epoch: 3 average loss: 0.00058404 total validation loss: 0.00007962
Epoch: 4 average loss: 0.00044413 total validation loss: 0.00005273
Epoch: 5 average loss: 0.00036411 total validation loss: 0.00007153
Epoch: 6 average loss: 0.00033399 total validation loss: 0.00011521
Epoch: 7 average loss: 0.00031604 total validation loss: 0.00011075
Epoch: 8 average loss: 0.00030774 total validation loss: 0.00006855
Epoch: 9 average loss: 0.00029670 total validation loss: 0.00011285
Epoch:10 average loss: 0.00028838 total validation loss: 0.00007529
Training Time : 1.6204872687657674 minutes 
total test loss of ['leaky', 'relu', 'elu', 'elu', 'elu']: [7.588134e-05]
total train loss of ['leaky', 'relu', 'elu', 'elu', '

total test loss of ['leaky', 'relu', 'elu', 'relu', 'elu']: [5.0746068e-05]
total train loss of ['leaky', 'relu', 'elu', 'relu', 'elu']: [5.1210798e-05]
[array([[0.352511  ],
       [0.4913545 ],
       [0.48858383],
       ...,
       [0.3748575 ],
       [0.13133734],
       [0.10493566]], dtype=float32)]
Average prediction time: 9.836713473002115
**********************Training 8 - ['leaky', 'relu', 'elu', 'relu', 'leaky']:
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00778433 total validation loss: 0.00056910
Epoch: 2 average loss: 0.00072086 total validation loss: 0.00020887
Epoch: 3 average loss: 0.00059693 total validation loss: 0.00009248
Epoch: 4 average loss: 0.00048493 total validation loss: 0.00019892
Epoch: 5 average loss: 0.00042639 total validation loss: 0.00009482
Epoch: 6 average loss: 0.00038541 total validation loss: 0.00009225
Epoch: 7 average loss: 0.00035280 total validation loss: 0.00023767
Epoch: 8 average los

Epoch: 6 average loss: 0.00035371 total validation loss: 0.00033730
Epoch: 7 average loss: 0.00031218 total validation loss: 0.00028696
Epoch: 8 average loss: 0.00028967 total validation loss: 0.00018922
Epoch: 9 average loss: 0.00027998 total validation loss: 0.00010204
Epoch:10 average loss: 0.00026455 total validation loss: 0.00023365
Training Time : 1.818222200870514 minutes 
total test loss of ['leaky', 'relu', 'leaky', 'leaky', 'leaky']: [0.00023404747]
total train loss of ['leaky', 'relu', 'leaky', 'leaky', 'leaky']: [0.00023462792]
[array([[0.35729718],
       [0.4763176 ],
       [0.48535702],
       ...,
       [0.38570094],
       [0.14311866],
       [0.12414392]], dtype=float32)]
Average prediction time: 10.582518577575684
**********************Training 15 - ['leaky', 'relu', 'leaky', 'leaky', 'relu']:
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00839085 total validation loss: 0.00023672
Epoch: 2 average loss: 0.000974

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00802863 total validation loss: 0.00030633
Epoch: 2 average loss: 0.00102045 total validation loss: 0.00024336
Epoch: 3 average loss: 0.00069586 total validation loss: 0.00009962
Epoch: 4 average loss: 0.00048668 total validation loss: 0.00008878
Epoch: 5 average loss: 0.00045512 total validation loss: 0.00013310
Epoch: 6 average loss: 0.00042838 total validation loss: 0.00008235
Epoch: 7 average loss: 0.00040480 total validation loss: 0.00010807
Epoch: 8 average loss: 0.00038696 total validation loss: 0.00010582
Epoch: 9 average loss: 0.00036693 total validation loss: 0.00004127
Epoch:10 average loss: 0.00035838 total validation loss: 0.00006512
Training Time : 1.9342960158983866 minutes 
total test loss of ['leaky', 'relu', 'relu', 'elu', 'relu']: [6.508511e-05]
total train loss of ['leaky', 'relu', 'relu', 'elu', 'relu']: [6.4991254e-05]
[array([[0.33733043],
       [0.48577508],
     

total test loss of ['leaky', 'relu', 'relu', 'relu', 'relu']: [0.00021519212]
total train loss of ['leaky', 'relu', 'relu', 'relu', 'relu']: [0.00021230025]
[array([[0.34209895],
       [0.4876983 ],
       [0.4867262 ],
       ...,
       [0.36963063],
       [0.14494042],
       [0.11125775]], dtype=float32)]
Average prediction time: 9.696384270985922


In [ ]:
#Second run -- 4 layers & 120 neurons & 'GBM' :
set_of_layers,training_time = training(4,120,'GBM')

**********************Training 1 - ('elu', 'elu', 'elu', 'elu'):
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00844334 total validation loss: 0.00153313
Epoch: 2 average loss: 0.00177885 total validation loss: 0.00128106
Epoch: 3 average loss: 0.00146059 total validation loss: 0.00052276
Epoch: 4 average loss: 0.00075539 total validation loss: 0.00023243
Epoch: 5 average loss: 0.00050625 total validation loss: 0.00027081
Epoch: 6 average loss: 0.00044128 total validation loss: 0.00022732
Epoch: 7 average loss: 0.00040235 total validation loss: 0.00025521
Epoch: 8 average loss: 0.00038641 total validation loss: 0.00011250
Epoch: 9 average loss: 0.00037320 total validation loss: 0.00016852
Epoch:10 average loss: 0.00036472 total validation loss: 0.00009314
Training Time : 1.1683355967203777 minutes 
total test loss of ('elu', 'elu', 'elu', 'elu'): [9.0776935e-05]
total train loss of ('elu', 'elu', 'elu', 'elu'): [9.213972e-05]
[array(

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00903471 total validation loss: 0.00076756
Epoch: 2 average loss: 0.00087164 total validation loss: 0.00020349
Epoch: 3 average loss: 0.00068901 total validation loss: 0.00014977
Epoch: 4 average loss: 0.00058799 total validation loss: 0.00020059
Epoch: 5 average loss: 0.00048346 total validation loss: 0.00022469
Epoch: 6 average loss: 0.00043541 total validation loss: 0.00016964
Epoch: 7 average loss: 0.00039391 total validation loss: 0.00024142
Epoch: 8 average loss: 0.00038085 total validation loss: 0.00021770
Epoch: 9 average loss: 0.00037639 total validation loss: 0.00024686
Epoch:10 average loss: 0.00035826 total validation loss: 0.00027467
Training Time : 1.222222618261973 minutes 
total test loss of ('elu', 'elu', 'relu', 'leaky'): [0.00027172908]
total train loss of ('elu', 'elu', 'relu', 'leaky'): [0.00027189535]
[array([[0.34686834],
       [0.45763874],
       [0.4575429 ],
  

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00908140 total validation loss: 0.00036590
Epoch: 2 average loss: 0.00099664 total validation loss: 0.00015533
Epoch: 3 average loss: 0.00061994 total validation loss: 0.00012183
Epoch: 4 average loss: 0.00043701 total validation loss: 0.00010345
Epoch: 5 average loss: 0.00039495 total validation loss: 0.00010854
Epoch: 6 average loss: 0.00036696 total validation loss: 0.00013967
Epoch: 7 average loss: 0.00034989 total validation loss: 0.00012165
Epoch: 8 average loss: 0.00033648 total validation loss: 0.00011602
Epoch: 9 average loss: 0.00032964 total validation loss: 0.00009315
Epoch:10 average loss: 0.00031805 total validation loss: 0.00007514
Training Time : 1.3402478138605753 minutes 
total test loss of ('elu', 'leaky', 'leaky', 'relu'): [7.4433134e-05]
total train loss of ('elu', 'leaky', 'leaky', 'relu'): [7.488726e-05]
[array([[0.34909675],
       [0.49752083],
       [0.49834248]

In [29]:
#Third run -- 4 layers & 20 neurons & 'GBM' :
set_of_layers,training_time = training(4,20,'GBM')

**********************Training 1 - ('elu', 'elu', 'elu', 'elu'):
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01731275 total validation loss: 0.00175889
Epoch: 2 average loss: 0.00353772 total validation loss: 0.00136327
Epoch: 3 average loss: 0.00237790 total validation loss: 0.00129831
Epoch: 4 average loss: 0.00205699 total validation loss: 0.00124796
Epoch: 5 average loss: 0.00189396 total validation loss: 0.00102351
Epoch: 6 average loss: 0.00180278 total validation loss: 0.00111998
Epoch: 7 average loss: 0.00174888 total validation loss: 0.00106331
Epoch: 8 average loss: 0.00173060 total validation loss: 0.00107209
Epoch: 9 average loss: 0.00170640 total validation loss: 0.00103824
Epoch:10 average loss: 0.00168775 total validation loss: 0.00108034
Training Time : 0.625614317258199 minutes 
total test loss of ('elu', 'elu', 'elu', 'elu'): [0.0010526364]
total train loss of ('elu', 'elu', 'elu', 'elu'): [0.0010473623]
[array([[

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.02056226 total validation loss: 0.00180647
Epoch: 2 average loss: 0.00338070 total validation loss: 0.00128923
Epoch: 3 average loss: 0.00288203 total validation loss: 0.00153485
Epoch: 4 average loss: 0.00261383 total validation loss: 0.00122773
Epoch: 5 average loss: 0.00231537 total validation loss: 0.00122807
Epoch: 6 average loss: 0.00211407 total validation loss: 0.00070451
Epoch: 7 average loss: 0.00172064 total validation loss: 0.00046813
Epoch: 8 average loss: 0.00158314 total validation loss: 0.00052362
Epoch: 9 average loss: 0.00152316 total validation loss: 0.00051857
Epoch:10 average loss: 0.00148631 total validation loss: 0.00057614
Training Time : 0.6547390858332316 minutes 
total test loss of ('elu', 'elu', 'relu', 'leaky'): [0.0005704929]
total train loss of ('elu', 'elu', 'relu', 'leaky'): [0.0005628573]
[array([[0.37646037],
       [0.16435751],
       [0.23476122],
   

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01984620 total validation loss: 0.00168825
Epoch: 2 average loss: 0.00329111 total validation loss: 0.00117099
Epoch: 3 average loss: 0.00222077 total validation loss: 0.00132638
Epoch: 4 average loss: 0.00211643 total validation loss: 0.00120746
Epoch: 5 average loss: 0.00205622 total validation loss: 0.00116342
Epoch: 6 average loss: 0.00203714 total validation loss: 0.00119315
Epoch: 7 average loss: 0.00186861 total validation loss: 0.00060199
Epoch: 8 average loss: 0.00162096 total validation loss: 0.00050506
Epoch: 9 average loss: 0.00158302 total validation loss: 0.00048271
Epoch:10 average loss: 0.00156536 total validation loss: 0.00050931
Training Time : 0.6756270011266072 minutes 
total test loss of ('elu', 'leaky', 'leaky', 'relu'): [0.00049978367]
total train loss of ('elu', 'leaky', 'leaky', 'relu'): [0.0004864441]
[array([[0.38127115],
       [0.15441065],
       [0.21086113]

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01766312 total validation loss: 0.00151496
Epoch: 2 average loss: 0.00308128 total validation loss: 0.00108810
Epoch: 3 average loss: 0.00206336 total validation loss: 0.00043571
Epoch: 4 average loss: 0.00144618 total validation loss: 0.00043025
Epoch: 5 average loss: 0.00128714 total validation loss: 0.00033807
Epoch: 6 average loss: 0.00123387 total validation loss: 0.00037020
Epoch: 7 average loss: 0.00121205 total validation loss: 0.00035847
Epoch: 8 average loss: 0.00119350 total validation loss: 0.00031087
Epoch: 9 average loss: 0.00118321 total validation loss: 0.00053256
Epoch:10 average loss: 0.00117165 total validation loss: 0.00044700
Training Time : 0.658829935391744 minutes 
total test loss of ('elu', 'relu', 'leaky', 'elu'): [0.0004424627]
total train loss of ('elu', 'relu', 'leaky', 'elu'): [0.00043887825]
[array([[0.3899745 ],
       [0.15118226],
       [0.22985005],
   

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01732121 total validation loss: 0.00055478
Epoch: 2 average loss: 0.00252235 total validation loss: 0.00055998
Epoch: 3 average loss: 0.00173764 total validation loss: 0.00053601
Epoch: 4 average loss: 0.00160916 total validation loss: 0.00040879
Epoch: 5 average loss: 0.00138202 total validation loss: 0.00031900
Epoch: 6 average loss: 0.00136046 total validation loss: 0.00049413
Epoch: 7 average loss: 0.00133572 total validation loss: 0.00038947
Epoch: 8 average loss: 0.00132635 total validation loss: 0.00042501
Epoch: 9 average loss: 0.00127675 total validation loss: 0.00036800
Epoch:10 average loss: 0.00123992 total validation loss: 0.00038414
Training Time : 0.682823097705841 minutes 
total test loss of ('leaky', 'elu', 'elu', 'leaky'): [0.00038119883]
total train loss of ('leaky', 'elu', 'elu', 'leaky'): [0.00037150993]
[array([[0.38091484],
       [0.14927015],
       [0.22125193],


...

Epoch:10 average loss: 0.00120993 total validation loss: 0.00043862
Training Time : 0.6894321997960409 minutes 
total test loss of ('leaky', 'relu', 'leaky', 'elu'): [0.0004325554]
total train loss of ('leaky', 'relu', 'leaky', 'elu'): [0.00042813766]
[array([[0.38814142],
       [0.15682301],
       [0.21817312],
       ...,
       [0.10894816],
       [0.06620336],
       [0.34806865]], dtype=float32)]
Average prediction time: 0.799719492594401
**********************Training 50 - ('leaky', 'relu', 'leaky', 'leaky'):
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01888065 total validation loss: 0.00091191
Epoch: 2 average loss: 0.00243897 total validation loss: 0.00038714
Epoch: 3 average loss: 0.00169505 total validation loss: 0.00044340
Epoch: 4 average loss: 0.00146869 total validation loss: 0.00033675
Epoch: 5 average loss: 0.00138657 total validation loss: 0.00031246
Epoch: 6 average loss: 0.00131832 total validation loss: 0.000

Epoch: 7 average loss: 0.00157636 total validation loss: 0.00052462
Epoch: 8 average loss: 0.00155576 total validation loss: 0.00047068
Epoch: 9 average loss: 0.00153669 total validation loss: 0.00036040
Epoch:10 average loss: 0.00152525 total validation loss: 0.00045926
Training Time : 0.6575363993644714 minutes 
total test loss of ('relu', 'elu', 'elu', 'leaky'): [0.00045663636]
total train loss of ('relu', 'elu', 'elu', 'leaky'): [0.00044259537]
[array([[0.38240325],
       [0.16185665],
       [0.19923663],
       ...,
       [0.09342526],
       [0.05423788],
       [0.32506073]], dtype=float32)]
Average prediction time: 0.8695205052693685
**********************Training 57 - ('relu', 'elu', 'elu', 'relu'):
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01923791 total validation loss: 0.00096003
Epoch: 2 average loss: 0.00343911 total validation loss: 0.00075452
Epoch: 3 average loss: 0.00250723 total validation loss: 0.00097898
E

Epoch: 4 average loss: 0.00284300 total validation loss: 0.00106391
Epoch: 5 average loss: 0.00283838 total validation loss: 0.00089136
Epoch: 6 average loss: 0.00283689 total validation loss: 0.00108143
Epoch: 7 average loss: 0.00284553 total validation loss: 0.00106619
Epoch: 8 average loss: 0.00281906 total validation loss: 0.00112180
Epoch: 9 average loss: 0.00282578 total validation loss: 0.00098339
Epoch:10 average loss: 0.00281492 total validation loss: 0.00107249
Training Time : 0.6298869808514913 minutes 
total test loss of ('relu', 'elu', 'relu', 'relu'): [0.0010528837]
total train loss of ('relu', 'elu', 'relu', 'relu'): [0.0010308314]
[array([[0.37482542],
       [0.14396329],
       [0.212093  ],
       ...,
       [0.07398707],
       [0.04173766],
       [0.3425191 ]], dtype=float32)]
Average prediction time: 0.8933345476786295
**********************Training 64 - ('relu', 'leaky', 'elu', 'elu'):
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Ep

Epoch: 1 average loss: 0.02010308 total validation loss: 0.00116730
Epoch: 2 average loss: 0.00326504 total validation loss: 0.00086093
Epoch: 3 average loss: 0.00232732 total validation loss: 0.00075294
Epoch: 4 average loss: 0.00176585 total validation loss: 0.00090033
Epoch: 5 average loss: 0.00164266 total validation loss: 0.00045013
Epoch: 6 average loss: 0.00160318 total validation loss: 0.00059643
Epoch: 7 average loss: 0.00159062 total validation loss: 0.00050124
Epoch: 8 average loss: 0.00158657 total validation loss: 0.00048643
Epoch: 9 average loss: 0.00158415 total validation loss: 0.00052763
Epoch:10 average loss: 0.00156560 total validation loss: 0.00044283
Training Time : 0.6607142448425293 minutes 
total test loss of ('relu', 'leaky', 'relu', 'elu'): [0.0004372563]
total train loss of ('relu', 'leaky', 'relu', 'elu'): [0.0004259578]
[array([[0.3861261 ],
       [0.15400562],
       [0.21312535],
       ...,
       [0.10653592],
       [0.05950379],
       [0.35442826]],

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01994307 total validation loss: 0.00129326
Epoch: 2 average loss: 0.00302000 total validation loss: 0.00053717
Epoch: 3 average loss: 0.00215021 total validation loss: 0.00057011
Epoch: 4 average loss: 0.00189302 total validation loss: 0.00047151
Epoch: 5 average loss: 0.00176514 total validation loss: 0.00048009
Epoch: 6 average loss: 0.00163682 total validation loss: 0.00039633
Epoch: 7 average loss: 0.00155461 total validation loss: 0.00040682
Epoch: 8 average loss: 0.00151132 total validation loss: 0.00026229
Epoch: 9 average loss: 0.00146692 total validation loss: 0.00046245
Epoch:10 average loss: 0.00146386 total validation loss: 0.00034500
Training Time : 0.6814582824707032 minutes 
total test loss of ('relu', 'relu', 'leaky', 'leaky'): [0.00033984712]
total train loss of ('relu', 'relu', 'leaky', 'leaky'): [0.00033244787]
[array([[0.39024127],
       [0.16404814],
       [0.223339

In [13]:
#Forth run -- 6 layers & 120 neurons & 'GBM' :
set_of_layers,training_time = training(6,120,'GBM')

**********************Training 1 - ['elu', 'elu', 'elu', 'elu', 'elu', 'elu']:
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01434041 total validation loss: 0.00138302
Epoch: 2 average loss: 0.00188550 total validation loss: 0.00128361
Epoch: 3 average loss: 0.00137297 total validation loss: 0.00022963
Epoch: 4 average loss: 0.00073010 total validation loss: 0.00030468
Epoch: 5 average loss: 0.00056963 total validation loss: 0.00017685
Epoch: 6 average loss: 0.00049783 total validation loss: 0.00033548
Epoch: 7 average loss: 0.00046204 total validation loss: 0.00017552
Epoch: 8 average loss: 0.00043501 total validation loss: 0.00018899
Epoch: 9 average loss: 0.00042043 total validation loss: 0.00010890
Epoch:10 average loss: 0.00041361 total validation loss: 0.00011679
Training Time : 1.5572643160820008 minutes 
total test loss of ['elu', 'elu', 'elu', 'elu', 'elu', 'elu']: [0.00011570946]
total train loss of ['elu', 'elu', 'elu', 'e

In [19]:
#Barrier Down & In:
#First run -- 2 layers & 120 neurons & 'BarrierDownIn' :
set_of_layers,training_time = training(2,120,'BarrierDownIn')

**********************Training 1 - ('elu', 'elu'):
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00971128 total validation loss: 0.00158321
Epoch: 2 average loss: 0.00190930 total validation loss: 0.00130501
Epoch: 3 average loss: 0.00135681 total validation loss: 0.00081294
Epoch: 4 average loss: 0.00103018 total validation loss: 0.00077737
Epoch: 5 average loss: 0.00089934 total validation loss: 0.00073717
Epoch: 6 average loss: 0.00069391 total validation loss: 0.00026198
Epoch: 7 average loss: 0.00033151 total validation loss: 0.00012565
Epoch: 8 average loss: 0.00025335 total validation loss: 0.00009329
Epoch: 9 average loss: 0.00022227 total validation loss: 0.00006751
Epoch:10 average loss: 0.00019938 total validation loss: 0.00007350
Training Time : 0.6964904308319092 minutes 
total test loss of ('elu', 'elu'): [7.120547e-05]
total train loss of ('elu', 'elu'): [6.9075235e-05]
[array([[0.23514643],
       [0.07953182],
      

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01186402 total validation loss: 0.00034181
Epoch: 2 average loss: 0.00063391 total validation loss: 0.00015532
Epoch: 3 average loss: 0.00052065 total validation loss: 0.00013100
Epoch: 4 average loss: 0.00044802 total validation loss: 0.00010411
Epoch: 5 average loss: 0.00040084 total validation loss: 0.00009859
Epoch: 6 average loss: 0.00034332 total validation loss: 0.00008582
Epoch: 7 average loss: 0.00029683 total validation loss: 0.00010113
Epoch: 8 average loss: 0.00026759 total validation loss: 0.00005946
Epoch: 9 average loss: 0.00025350 total validation loss: 0.00005794
Epoch:10 average loss: 0.00024315 total validation loss: 0.00007434
Training Time : 0.7566131472587585 minutes 
total test loss of ('relu', 'leaky'): [7.289786e-05]
total train loss of ('relu', 'leaky'): [7.092883e-05]
[array([[2.3290657e-01],
       [7.3198237e-02],
       [2.3529923e-01],
       ...,
       [2.

In [20]:
#Barrier Down & In:
#First run -- 4 layers & 120 neurons & 'BarrierDownIn' :
set_of_layers,training_time = training(4,120,'BarrierDownIn')

**********************Training 1 - ('elu', 'elu', 'elu', 'elu'):
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01095778 total validation loss: 0.00146281
Epoch: 2 average loss: 0.00132713 total validation loss: 0.00076846
Epoch: 3 average loss: 0.00110226 total validation loss: 0.00091016
Epoch: 4 average loss: 0.00092997 total validation loss: 0.00022159
Epoch: 5 average loss: 0.00041901 total validation loss: 0.00011485
Epoch: 6 average loss: 0.00030391 total validation loss: 0.00020606
Epoch: 7 average loss: 0.00026247 total validation loss: 0.00007958
Epoch: 8 average loss: 0.00024825 total validation loss: 0.00010406
Epoch: 9 average loss: 0.00023345 total validation loss: 0.00027219
Epoch:10 average loss: 0.00023084 total validation loss: 0.00010282
Training Time : 1.2691203157107036 minutes 
total test loss of ('elu', 'elu', 'elu', 'elu'): [9.950164e-05]
total train loss of ('elu', 'elu', 'elu', 'elu'): [9.773416e-05]
[array([

[array([[2.3257865e-01],
       [8.2920626e-02],
       [2.3415688e-01],
       ...,
       [2.6232141e-01],
       [1.8180843e-05],
       [3.1395294e-02]], dtype=float32)]
Average prediction time: 8.023416996002197
**********************Training 8 - ('elu', 'elu', 'relu', 'leaky'):
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00943251 total validation loss: 0.00069623
Epoch: 2 average loss: 0.00069308 total validation loss: 0.00011384
Epoch: 3 average loss: 0.00045517 total validation loss: 0.00007184
Epoch: 4 average loss: 0.00038731 total validation loss: 0.00006318
Epoch: 5 average loss: 0.00033208 total validation loss: 0.00012704
Epoch: 6 average loss: 0.00028480 total validation loss: 0.00007059
Epoch: 7 average loss: 0.00025144 total validation loss: 0.00010276
Epoch: 8 average loss: 0.00022996 total validation loss: 0.00013141
Epoch: 9 average loss: 0.00021929 total validation loss: 0.00013502
Epoch:10 average loss: 0.0002

Epoch:10 average loss: 0.00020006 total validation loss: 0.00006045
Training Time : 1.3076215147972108 minutes 
total test loss of ('elu', 'leaky', 'leaky', 'leaky'): [5.8660204e-05]
total train loss of ('elu', 'leaky', 'leaky', 'leaky'): [5.6818633e-05]
[array([[2.3973797e-01],
       [7.6066807e-02],
       [2.2510493e-01],
       ...,
       [2.6643348e-01],
       [2.2160613e-04],
       [2.8454684e-02]], dtype=float32)]
Average prediction time: 7.708017031351726
**********************Training 15 - ('elu', 'leaky', 'leaky', 'relu'):
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01036210 total validation loss: 0.00023696
Epoch: 2 average loss: 0.00072245 total validation loss: 0.00013713
Epoch: 3 average loss: 0.00050511 total validation loss: 0.00009255
Epoch: 4 average loss: 0.00032904 total validation loss: 0.00005237
Epoch: 5 average loss: 0.00026142 total validation loss: 0.00007292
Epoch: 6 average loss: 0.00024391 total val

Epoch: 5 average loss: 0.00032596 total validation loss: 0.00009313
Epoch: 6 average loss: 0.00031254 total validation loss: 0.00010203
Epoch: 7 average loss: 0.00029364 total validation loss: 0.00011893
Epoch: 8 average loss: 0.00028377 total validation loss: 0.00008104
Epoch: 9 average loss: 0.00026968 total validation loss: 0.00009601
Epoch:10 average loss: 0.00026323 total validation loss: 0.00007952
Training Time : 1.2488072554270426 minutes 
total test loss of ('elu', 'relu', 'elu', 'relu'): [7.806341e-05]
total train loss of ('elu', 'relu', 'elu', 'relu'): [7.5929485e-05]
[array([[2.3491517e-01],
       [7.3308438e-02],
       [2.3221938e-01],
       ...,
       [2.6473954e-01],
       [1.7072678e-05],
       [2.7316168e-02]], dtype=float32)]
Average prediction time: 6.9214661916097
**********************Training 22 - ('elu', 'relu', 'leaky', 'elu'):
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00885441 total validation loss:

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00873093 total validation loss: 0.00077324
Epoch: 2 average loss: 0.00060555 total validation loss: 0.00017058
Epoch: 3 average loss: 0.00054836 total validation loss: 0.00028249
Epoch: 4 average loss: 0.00049148 total validation loss: 0.00018742
Epoch: 5 average loss: 0.00042155 total validation loss: 0.00014277
Epoch: 6 average loss: 0.00035559 total validation loss: 0.00016725
Epoch: 7 average loss: 0.00030305 total validation loss: 0.00017807
Epoch: 8 average loss: 0.00025575 total validation loss: 0.00005841
Epoch: 9 average loss: 0.00022875 total validation loss: 0.00021664
Epoch:10 average loss: 0.00022438 total validation loss: 0.00012614
Training Time : 1.3180835843086243 minutes 
total test loss of ('leaky', 'elu', 'elu', 'elu'): [0.00012556447]
total train loss of ('leaky', 'elu', 'elu', 'elu'): [0.00012496935]
[array([[0.22931165],
       [0.07607704],
       [0.21930465],
   

...

Epoch: 4 average loss: 0.00040378 total validation loss: 0.00004738
Epoch: 5 average loss: 0.00032739 total validation loss: 0.00005138
Epoch: 6 average loss: 0.00031534 total validation loss: 0.00008866
Epoch: 7 average loss: 0.00030166 total validation loss: 0.00009536
Epoch: 8 average loss: 0.00029407 total validation loss: 0.00005481
Epoch: 9 average loss: 0.00027690 total validation loss: 0.00004422
Epoch:10 average loss: 0.00026946 total validation loss: 0.00003862
Training Time : 1.1710363507270813 minutes 
total test loss of ('leaky', 'relu', 'elu', 'relu'): [3.870887e-05]
total train loss of ('leaky', 'relu', 'elu', 'relu'): [3.7242902e-05]
[array([[2.4560270e-01],
       [7.7461474e-02],
       [2.4041033e-01],
       ...,
       [2.7293020e-01],
       [2.9108012e-05],
       [2.8907273e-02]], dtype=float32)]
Average prediction time: 6.5320173899332685
**********************Training 49 - ('leaky', 'relu', 'leaky', 'elu'):
Train on 216000 samples, validate on 24000 samples, t

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00858653 total validation loss: 0.00025530
Epoch: 2 average loss: 0.00061149 total validation loss: 0.00011719
Epoch: 3 average loss: 0.00050153 total validation loss: 0.00017326
Epoch: 4 average loss: 0.00041172 total validation loss: 0.00016267
Epoch: 5 average loss: 0.00033197 total validation loss: 0.00006556
Epoch: 6 average loss: 0.00028795 total validation loss: 0.00010507
Epoch: 7 average loss: 0.00026748 total validation loss: 0.00007968
Epoch: 8 average loss: 0.00025352 total validation loss: 0.00007442
Epoch: 9 average loss: 0.00024595 total validation loss: 0.00005344
Epoch:10 average loss: 0.00023999 total validation loss: 0.00007929
Training Time : 1.3180118163426717 minutes 
total test loss of ('relu', 'elu', 'elu', 'elu'): [7.851839e-05]
total train loss of ('relu', 'elu', 'elu', 'elu'): [7.6267024e-05]
[array([[0.23104016],
       [0.07630858],
       [0.24142712],
      

[array([[0.24110703],
       [0.08231759],
       [0.24070828],
       ...,
       [0.2556539 ],
       [0.00110822],
       [0.02990287]], dtype=float32)]
Average prediction time: 7.294515768686931
**********************Training 62 - ('relu', 'elu', 'relu', 'leaky'):
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00952301 total validation loss: 0.00019245
Epoch: 2 average loss: 0.00063185 total validation loss: 0.00008088
Epoch: 3 average loss: 0.00055019 total validation loss: 0.00013448
Epoch: 4 average loss: 0.00048630 total validation loss: 0.00009504
Epoch: 5 average loss: 0.00042615 total validation loss: 0.00009618
Epoch: 6 average loss: 0.00037818 total validation loss: 0.00011492
Epoch: 7 average loss: 0.00036600 total validation loss: 0.00008971
Epoch: 8 average loss: 0.00033775 total validation loss: 0.00010731
Epoch: 9 average loss: 0.00031076 total validation loss: 0.00012195
Epoch:10 average loss: 0.00029630 total valid

Epoch: 9 average loss: 0.00022393 total validation loss: 0.00011892
Epoch:10 average loss: 0.00021350 total validation loss: 0.00008333
Training Time : 1.186475662390391 minutes 
total test loss of ('relu', 'leaky', 'leaky', 'leaky'): [8.224314e-05]
total train loss of ('relu', 'leaky', 'leaky', 'leaky'): [8.0469494e-05]
[array([[2.4727412e-01],
       [8.4702305e-02],
       [2.4387199e-01],
       ...,
       [2.7117601e-01],
       [6.5216263e-06],
       [3.5735723e-02]], dtype=float32)]
Average prediction time: 7.367599010467529
**********************Training 69 - ('relu', 'leaky', 'leaky', 'relu'):
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01050267 total validation loss: 0.00022250
Epoch: 2 average loss: 0.00083902 total validation loss: 0.00009468
Epoch: 3 average loss: 0.00061977 total validation loss: 0.00007221
Epoch: 4 average loss: 0.00039645 total validation loss: 0.00014691
Epoch: 5 average loss: 0.00033890 total va

Epoch: 4 average loss: 0.00047665 total validation loss: 0.00012182
Epoch: 5 average loss: 0.00041112 total validation loss: 0.00007989
Epoch: 6 average loss: 0.00038746 total validation loss: 0.00005906
Epoch: 7 average loss: 0.00036352 total validation loss: 0.00011195
Epoch: 8 average loss: 0.00034394 total validation loss: 0.00006484
Epoch: 9 average loss: 0.00032823 total validation loss: 0.00006939
Epoch:10 average loss: 0.00031383 total validation loss: 0.00014681
Training Time : 1.1085421363512675 minutes 
total test loss of ('relu', 'relu', 'elu', 'relu'): [0.00014472094]
total train loss of ('relu', 'relu', 'elu', 'relu'): [0.00014228297]
[array([[2.5791174e-01],
       [7.6932274e-02],
       [2.5081959e-01],
       ...,
       [2.8795201e-01],
       [1.4946912e-05],
       [3.4109183e-02]], dtype=float32)]
Average prediction time: 7.313048839569092
**********************Training 76 - ('relu', 'relu', 'leaky', 'elu'):
Train on 216000 samples, validate on 24000 samples, test

In [24]:
#Barrier Up & In:
#First run -- 2 layers & 120 neurons & 'BarrierUpIn' :
set_of_layers,training_time = training(2,120,'BarrierUpIn')

**********************Training 1 - ('elu', 'elu'):
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00907299 total validation loss: 0.00193446
Epoch: 2 average loss: 0.00228097 total validation loss: 0.00144581
Epoch: 3 average loss: 0.00172574 total validation loss: 0.00117535
Epoch: 4 average loss: 0.00133954 total validation loss: 0.00054322
Epoch: 5 average loss: 0.00066832 total validation loss: 0.00035373
Epoch: 6 average loss: 0.00046135 total validation loss: 0.00018567
Epoch: 7 average loss: 0.00038040 total validation loss: 0.00012670
Epoch: 8 average loss: 0.00034536 total validation loss: 0.00013716
Epoch: 9 average loss: 0.00032521 total validation loss: 0.00018120
Epoch:10 average loss: 0.00031165 total validation loss: 0.00016999
Training Time : 0.7898387829462687 minutes 
total test loss of ('elu', 'elu'): [0.00016702943]
total train loss of ('elu', 'elu'): [0.00016766011]
[array([[0.21078108],
       [0.2914697 ],
     

Epoch: 1 average loss: 0.01094588 total validation loss: 0.00035223
Epoch: 2 average loss: 0.00094200 total validation loss: 0.00022850
Epoch: 3 average loss: 0.00081339 total validation loss: 0.00025574
Epoch: 4 average loss: 0.00071865 total validation loss: 0.00018992
Epoch: 5 average loss: 0.00061957 total validation loss: 0.00021673
Epoch: 6 average loss: 0.00052259 total validation loss: 0.00018568
Epoch: 7 average loss: 0.00046909 total validation loss: 0.00013484
Epoch: 8 average loss: 0.00044262 total validation loss: 0.00012397
Epoch: 9 average loss: 0.00042907 total validation loss: 0.00014559
Epoch:10 average loss: 0.00041033 total validation loss: 0.00011876
Training Time : 0.9270714004834493 minutes 
total test loss of ('relu', 'leaky'): [0.00011686017]
total train loss of ('relu', 'leaky'): [0.000116821924]
[array([[0.20443487],
       [0.28656828],
       [0.53656036],
       ...,
       [0.050548  ],
       [0.32862467],
       [0.5781982 ]], dtype=float32)]
Average pr

In [32]:
#Barrier Up & In:
#First run -- 4 layers & 120 neurons & 'BarrierUpIn' :
set_of_layers,training_time = training(4,120,'BarrierUpIn')

**********************Training 1 - ('elu', 'elu', 'elu', 'elu'):
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00853708 total validation loss: 0.00127411
Epoch: 2 average loss: 0.00181830 total validation loss: 0.00126034
Epoch: 3 average loss: 0.00141667 total validation loss: 0.00060215
Epoch: 4 average loss: 0.00070216 total validation loss: 0.00016406
Epoch: 5 average loss: 0.00050430 total validation loss: 0.00012636
Epoch: 6 average loss: 0.00043779 total validation loss: 0.00015211
Epoch: 7 average loss: 0.00039943 total validation loss: 0.00012926
Epoch: 8 average loss: 0.00038432 total validation loss: 0.00014299
Epoch: 9 average loss: 0.00037528 total validation loss: 0.00010078
Epoch:10 average loss: 0.00036254 total validation loss: 0.00016159
Training Time : 1.1147174835205078 minutes 
total test loss of ('elu', 'elu', 'elu', 'elu'): [0.00016078481]
total train loss of ('elu', 'elu', 'elu', 'elu'): [0.00016089698]
[array

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00891696 total validation loss: 0.00084734
Epoch: 2 average loss: 0.00080353 total validation loss: 0.00014062
Epoch: 3 average loss: 0.00060976 total validation loss: 0.00019432
Epoch: 4 average loss: 0.00049457 total validation loss: 0.00020646
Epoch: 5 average loss: 0.00043908 total validation loss: 0.00012543
Epoch: 6 average loss: 0.00039925 total validation loss: 0.00016284
Epoch: 7 average loss: 0.00036416 total validation loss: 0.00030168
Epoch: 8 average loss: 0.00035002 total validation loss: 0.00026859
Epoch: 9 average loss: 0.00034108 total validation loss: 0.00043403
Epoch:10 average loss: 0.00033774 total validation loss: 0.00022317
Training Time : 1.3680052479108176 minutes 
total test loss of ('elu', 'elu', 'relu', 'leaky'): [0.00022313865]
total train loss of ('elu', 'elu', 'relu', 'leaky'): [0.00022303335]
[array([[0.2078967 ],
       [0.27798882],
       [0.490686  ],
 

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00915244 total validation loss: 0.00026508
Epoch: 2 average loss: 0.00104462 total validation loss: 0.00017074
Epoch: 3 average loss: 0.00064420 total validation loss: 0.00014840
Epoch: 4 average loss: 0.00043548 total validation loss: 0.00011843
Epoch: 5 average loss: 0.00039897 total validation loss: 0.00013806
Epoch: 6 average loss: 0.00038672 total validation loss: 0.00010939
Epoch: 7 average loss: 0.00037261 total validation loss: 0.00009813
Epoch: 8 average loss: 0.00036868 total validation loss: 0.00010481
Epoch: 9 average loss: 0.00035854 total validation loss: 0.00013049
Epoch:10 average loss: 0.00034387 total validation loss: 0.00009474
Training Time : 1.3405235330263774 minutes 
total test loss of ('elu', 'leaky', 'leaky', 'relu'): [9.273994e-05]
total train loss of ('elu', 'leaky', 'leaky', 'relu'): [9.4019895e-05]
[array([[0.19098967],
       [0.27419183],
       [0.51775014]

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00735849 total validation loss: 0.00022220
Epoch: 2 average loss: 0.00069449 total validation loss: 0.00021966
Epoch: 3 average loss: 0.00053582 total validation loss: 0.00021149
Epoch: 4 average loss: 0.00042655 total validation loss: 0.00021378
Epoch: 5 average loss: 0.00035291 total validation loss: 0.00035500
Epoch: 6 average loss: 0.00030734 total validation loss: 0.00023828
Epoch: 7 average loss: 0.00029498 total validation loss: 0.00023312
Epoch: 8 average loss: 0.00027826 total validation loss: 0.00018001
Epoch: 9 average loss: 0.00027242 total validation loss: 0.00014660
Epoch:10 average loss: 0.00026071 total validation loss: 0.00016311
Training Time : 1.3025651017824809 minutes 
total test loss of ('elu', 'relu', 'leaky', 'elu'): [0.00016028539]
total train loss of ('elu', 'relu', 'leaky', 'elu'): [0.00016134545]
[array([[0.20078886],
       [0.28152448],
       [0.50206935],
 

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00754557 total validation loss: 0.00025832
Epoch: 2 average loss: 0.00086470 total validation loss: 0.00025922
Epoch: 3 average loss: 0.00066271 total validation loss: 0.00016886
Epoch: 4 average loss: 0.00056634 total validation loss: 0.00019173
Epoch: 5 average loss: 0.00047868 total validation loss: 0.00015867
Epoch: 6 average loss: 0.00045941 total validation loss: 0.00024841
Epoch: 7 average loss: 0.00041901 total validation loss: 0.00018129
Epoch: 8 average loss: 0.00039973 total validation loss: 0.00009393
Epoch: 9 average loss: 0.00037499 total validation loss: 0.00013685
Epoch:10 average loss: 0.00035594 total validation loss: 0.00007914
Training Time : 1.2797074516614277 minutes 
total test loss of ('leaky', 'elu', 'elu', 'leaky'): [7.61566e-05]
total train loss of ('leaky', 'elu', 'elu', 'leaky'): [7.7064644e-05]
[array([[0.19431543],
       [0.27881044],
       [0.53892165],
 

...

Epoch:10 average loss: 0.00023858 total validation loss: 0.00023459
Training Time : 1.282821516195933 minutes 
total test loss of ('leaky', 'relu', 'leaky', 'elu'): [0.00023594758]
total train loss of ('leaky', 'relu', 'leaky', 'elu'): [0.00023396553]
[array([[0.19437033],
       [0.29231665],
       [0.5519486 ],
       ...,
       [0.04635426],
       [0.3494776 ],
       [0.5872148 ]], dtype=float32)]
Average prediction time: 7.942716280619304
**********************Training 50 - ('leaky', 'relu', 'leaky', 'leaky'):
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00771034 total validation loss: 0.00024753
Epoch: 2 average loss: 0.00067163 total validation loss: 0.00028204
Epoch: 3 average loss: 0.00050705 total validation loss: 0.00008250
Epoch: 4 average loss: 0.00039104 total validation loss: 0.00031393
Epoch: 5 average loss: 0.00033786 total validation loss: 0.00020229
Epoch: 6 average loss: 0.00030425 total validation loss: 0.000

Epoch: 7 average loss: 0.00043211 total validation loss: 0.00013210
Epoch: 8 average loss: 0.00040946 total validation loss: 0.00013599
Epoch: 9 average loss: 0.00039402 total validation loss: 0.00008201
Epoch:10 average loss: 0.00038376 total validation loss: 0.00010065
Training Time : 1.234842300415039 minutes 
total test loss of ('relu', 'elu', 'elu', 'leaky'): [9.906942e-05]
total train loss of ('relu', 'elu', 'elu', 'leaky'): [9.88736e-05]
[array([[0.19288132],
       [0.2849874 ],
       [0.5334003 ],
       ...,
       [0.05746461],
       [0.31946546],
       [0.56142235]], dtype=float32)]
Average prediction time: 10.186580816904703
**********************Training 57 - ('relu', 'elu', 'elu', 'relu'):
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00795749 total validation loss: 0.00038025
Epoch: 2 average loss: 0.00104801 total validation loss: 0.00028209
Epoch: 3 average loss: 0.00078496 total validation loss: 0.00014176
Epoch

Epoch: 4 average loss: 0.00060191 total validation loss: 0.00023696
Epoch: 5 average loss: 0.00057514 total validation loss: 0.00017257
Epoch: 6 average loss: 0.00055892 total validation loss: 0.00015147
Epoch: 7 average loss: 0.00054291 total validation loss: 0.00015038
Epoch: 8 average loss: 0.00052683 total validation loss: 0.00018535
Epoch: 9 average loss: 0.00051749 total validation loss: 0.00015645
Epoch:10 average loss: 0.00051398 total validation loss: 0.00016044
Training Time : 1.165156869093577 minutes 
total test loss of ('relu', 'elu', 'relu', 'relu'): [0.0001585566]
total train loss of ('relu', 'elu', 'relu', 'relu'): [0.00015754893]
[array([[0.19451651],
       [0.28769404],
       [0.52451944],
       ...,
       [0.05110439],
       [0.32161224],
       [0.54554075]], dtype=float32)]
Average prediction time: 6.956470012664795
**********************Training 64 - ('relu', 'leaky', 'elu', 'elu'):
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epo

Epoch: 1 average loss: 0.00787941 total validation loss: 0.00031169
Epoch: 2 average loss: 0.00080544 total validation loss: 0.00014248
Epoch: 3 average loss: 0.00069800 total validation loss: 0.00040670
Epoch: 4 average loss: 0.00055883 total validation loss: 0.00016380
Epoch: 5 average loss: 0.00045129 total validation loss: 0.00009836
Epoch: 6 average loss: 0.00040200 total validation loss: 0.00012414
Epoch: 7 average loss: 0.00036581 total validation loss: 0.00031342
Epoch: 8 average loss: 0.00034974 total validation loss: 0.00021944
Epoch: 9 average loss: 0.00033502 total validation loss: 0.00010812
Epoch:10 average loss: 0.00032776 total validation loss: 0.00022547
Training Time : 1.2289586464564006 minutes 
total test loss of ('relu', 'leaky', 'relu', 'elu'): [0.00022350789]
total train loss of ('relu', 'leaky', 'relu', 'elu'): [0.0002223239]
[array([[0.20985566],
       [0.29827592],
       [0.5148422 ],
       ...,
       [0.04994088],
       [0.34356272],
       [0.55043596]]

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00809540 total validation loss: 0.00018036
Epoch: 2 average loss: 0.00083466 total validation loss: 0.00018718
Epoch: 3 average loss: 0.00070045 total validation loss: 0.00017823
Epoch: 4 average loss: 0.00055940 total validation loss: 0.00012277
Epoch: 5 average loss: 0.00048579 total validation loss: 0.00015942
Epoch: 6 average loss: 0.00045681 total validation loss: 0.00016512
Epoch: 7 average loss: 0.00042193 total validation loss: 0.00039048
Epoch: 8 average loss: 0.00039453 total validation loss: 0.00060335
Epoch: 9 average loss: 0.00036363 total validation loss: 0.00060911
Epoch:10 average loss: 0.00034129 total validation loss: 0.00045724
Training Time : 1.2095580299695332 minutes 
total test loss of ('relu', 'relu', 'leaky', 'leaky'): [0.00045210813]
total train loss of ('relu', 'relu', 'leaky', 'leaky'): [0.000453828]
[array([[0.22709915],
       [0.29452252],
       [0.54292727

In [14]:
#Data load:
model = 'GBM'
X_train=np.loadtxt('X_train_1_%s'% model)
y_train=np.loadtxt('y_train_1_%s'% model)
y_train=y_train.reshape(len(y_train),1)
X_test=np.loadtxt('X_test_1_%s'% model)
y_test=np.loadtxt('y_test_1_%s'% model)
y_test=y_test.reshape(len(y_test),1)
X_val=np.loadtxt('X_val_1_%s'% model)
y_val=np.loadtxt('y_val_1_%s'% model)
y_val=y_val.reshape(len(y_val),1)
X_data_H=np.loadtxt('X_data_H_1_%s'% model)
y_data_H=np.loadtxt('y_data_H_1_%s'% model)
y_data_H=y_data_H.reshape(len(y_data_H),1)

In [17]:
#First run -- 2 layers & 'GBM' :
for j in np.arange(20,120,20):
    print("%%%%%%%%%%%%%%%%% Training 2 Layers & {} Neurons:".format(j))
    set_of_layers,training_time = training(2,j,'GBM')

%%%%%%%%%%%%%%%%% Training 2 Layers & 20 Neurons:
**********************Training 1 - ('elu', 'elu'):
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01771374 total validation loss: 0.00158104
Epoch: 2 average loss: 0.00164020 total validation loss: 0.00157982
Epoch: 3 average loss: 0.00122627 total validation loss: 0.00016037
Epoch: 4 average loss: 0.00030568 total validation loss: 0.00016565
Epoch: 5 average loss: 0.00024103 total validation loss: 0.00009278
Epoch: 6 average loss: 0.00019379 total validation loss: 0.00008238
Epoch: 7 average loss: 0.00014662 total validation loss: 0.00005736
Epoch: 8 average loss: 0.00010147 total validation loss: 0.00004189
Epoch: 9 average loss: 0.00007422 total validation loss: 0.00003760
Epoch:10 average loss: 0.00006439 total validation loss: 0.00003671
Training Time : 0.4788576324780782 minutes 
total test loss of ('elu', 'elu'): [3.4704306e-05]
total train loss of ('elu', 'elu'): [3.472489e-05]

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.02067082 total validation loss: 0.00158965
Epoch: 2 average loss: 0.00165176 total validation loss: 0.00158751
Epoch: 3 average loss: 0.00164240 total validation loss: 0.00158763
Epoch: 4 average loss: 0.00164180 total validation loss: 0.00158759
Epoch: 5 average loss: 0.00048194 total validation loss: 0.00004353
Epoch: 6 average loss: 0.00011372 total validation loss: 0.00002913
Epoch: 7 average loss: 0.00008102 total validation loss: 0.00002517
Epoch: 8 average loss: 0.00006678 total validation loss: 0.00002340
Epoch: 9 average loss: 0.00005687 total validation loss: 0.00001673
Epoch:10 average loss: 0.00005597 total validation loss: 0.00001883
Training Time : 0.49509650468826294 minutes 
total test loss of ('relu', 'leaky'): [1.9591702e-05]
total train loss of ('relu', 'leaky'): [1.9342164e-05]
[array([[1.8414101e-04],
       [2.0123736e-03],
       [1.5309455e-05],
       ...,
       

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01700989 total validation loss: 0.00158803
Epoch: 2 average loss: 0.00164764 total validation loss: 0.00158764
Epoch: 3 average loss: 0.00163842 total validation loss: 0.00158768
Epoch: 4 average loss: 0.00163843 total validation loss: 0.00158762
Epoch: 5 average loss: 0.00041940 total validation loss: 0.00002378
Epoch: 6 average loss: 0.00007306 total validation loss: 0.00001747
Epoch: 7 average loss: 0.00004370 total validation loss: 0.00001343
Epoch: 8 average loss: 0.00004176 total validation loss: 0.00001496
Epoch: 9 average loss: 0.00004123 total validation loss: 0.00001419
Epoch:10 average loss: 0.00004095 total validation loss: 0.00000944
Training Time : 0.5330169359842937 minutes 
total test loss of ('leaky', 'relu'): [1.0023246e-05]
total train loss of ('leaky', 'relu'): [9.61347e-06]
[array([[2.1232980e-04],
       [3.3176281e-03],
       [2.6469552e-05],
       ...,
       [7.

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01181440 total validation loss: 0.00158536
Epoch: 2 average loss: 0.00161936 total validation loss: 0.00126847
Epoch: 3 average loss: 0.00015841 total validation loss: 0.00001466
Epoch: 4 average loss: 0.00007278 total validation loss: 0.00001905
Epoch: 5 average loss: 0.00005972 total validation loss: 0.00001419
Epoch: 6 average loss: 0.00004718 total validation loss: 0.00000890
Epoch: 7 average loss: 0.00003667 total validation loss: 0.00000796
Epoch: 8 average loss: 0.00003072 total validation loss: 0.00000795
Epoch: 9 average loss: 0.00002715 total validation loss: 0.00001488
Epoch:10 average loss: 0.00002544 total validation loss: 0.00000860
Training Time : 0.5797195355097453 minutes 
total test loss of ('leaky', 'elu'): [8.317509e-06]
total train loss of ('leaky', 'elu'): [8.3767545e-06]
[array([[8.5279380e-04],
       [6.9112643e-03],
       [2.2764070e-04],
       ...,
       [1.2

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01068347 total validation loss: 0.00158851
Epoch: 2 average loss: 0.00163816 total validation loss: 0.00158549
Epoch: 3 average loss: 0.00064982 total validation loss: 0.00009990
Epoch: 4 average loss: 0.00015949 total validation loss: 0.00010702
Epoch: 5 average loss: 0.00013535 total validation loss: 0.00005980
Epoch: 6 average loss: 0.00007589 total validation loss: 0.00005178
Epoch: 7 average loss: 0.00006428 total validation loss: 0.00004747
Epoch: 8 average loss: 0.00005041 total validation loss: 0.00001805
Epoch: 9 average loss: 0.00003939 total validation loss: 0.00001452
Epoch:10 average loss: 0.00003290 total validation loss: 0.00002179
Training Time : 0.6204683144887289 minutes 
total test loss of ('elu', 'leaky'): [2.1421314e-05]
total train loss of ('elu', 'leaky'): [2.1547497e-05]
[array([[0.00129437],
       [0.00696011],
       [0.00040819],
       ...,
       [0.01277927]

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01355658 total validation loss: 0.00158768
Epoch: 2 average loss: 0.00164058 total validation loss: 0.00158769
Epoch: 3 average loss: 0.00163941 total validation loss: 0.00158769
Epoch: 4 average loss: 0.00089269 total validation loss: 0.00003094
Epoch: 5 average loss: 0.00008574 total validation loss: 0.00001347
Epoch: 6 average loss: 0.00005780 total validation loss: 0.00001250
Epoch: 7 average loss: 0.00004159 total validation loss: 0.00000771
Epoch: 8 average loss: 0.00003398 total validation loss: 0.00000939
Epoch: 9 average loss: 0.00003283 total validation loss: 0.00000704
Epoch:10 average loss: 0.00003258 total validation loss: 0.00000745
Training Time : 0.5667924205462138 minutes 
total test loss of ('relu', 'relu'): [8.137671e-06]
total train loss of ('relu', 'relu'): [7.785729e-06]
[array([[3.5578327e-04],
       [1.2971293e-03],
       [2.3673292e-05],
       ...,
       [2.82

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01068143 total validation loss: 0.00158299
Epoch: 2 average loss: 0.00140796 total validation loss: 0.00006675
Epoch: 3 average loss: 0.00007491 total validation loss: 0.00001494
Epoch: 4 average loss: 0.00006000 total validation loss: 0.00001101
Epoch: 5 average loss: 0.00005232 total validation loss: 0.00000748
Epoch: 6 average loss: 0.00004479 total validation loss: 0.00001197
Epoch: 7 average loss: 0.00003874 total validation loss: 0.00001738
Epoch: 8 average loss: 0.00003394 total validation loss: 0.00000730
Epoch: 9 average loss: 0.00003125 total validation loss: 0.00000682
Epoch:10 average loss: 0.00002846 total validation loss: 0.00001072
Training Time : 0.6416133165359497 minutes 
total test loss of ('relu', 'elu'): [1.1086891e-05]
total train loss of ('relu', 'elu'): [1.0961324e-05]
[array([[5.5954501e-04],
       [1.3927984e-03],
       [5.2456417e-05],
       ...,
       [2.52

In [212]:
#7th run -- 4 layers & 20 neurons & 'GBM':
set_of_layers,training_time = training(4,20,'GBM')

**********************Training 1 - ('elu', 'elu', 'leaky', 'relu'):
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.02586141 total validation loss: 0.00174840
Epoch: 2 average loss: 0.00400322 total validation loss: 0.00194648
Epoch: 3 average loss: 0.00370511 total validation loss: 0.00196571
Epoch: 4 average loss: 0.00350656 total validation loss: 0.00194705
Epoch: 5 average loss: 0.00324881 total validation loss: 0.00166040
Epoch: 6 average loss: 0.00317584 total validation loss: 0.00178527
Epoch: 7 average loss: 0.00317415 total validation loss: 0.00188637
Epoch: 8 average loss: 0.00314522 total validation loss: 0.00183176
Epoch: 9 average loss: 0.00315112 total validation loss: 0.00177549
Epoch:10 average loss: 0.00316545 total validation loss: 0.00174697
Training Time : 0.816284449895223 minutes 
total test loss of ('elu', 'elu', 'leaky', 'relu'): [0.0017487938]
total train loss of ('elu', 'elu', 'leaky', 'relu'): [0.0017622912]


Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.05456386 total validation loss: 0.04070015
Epoch: 2 average loss: 0.04300619 total validation loss: 0.04063366
Epoch: 3 average loss: 0.04116802 total validation loss: 0.04052289
Epoch: 4 average loss: 0.04093657 total validation loss: 0.04054625
Epoch: 5 average loss: 0.00953600 total validation loss: 0.00231088
Epoch: 6 average loss: 0.00455224 total validation loss: 0.00235374
Epoch: 7 average loss: 0.00426652 total validation loss: 0.00208216
Epoch: 8 average loss: 0.00408665 total validation loss: 0.00202644
Epoch: 9 average loss: 0.00395271 total validation loss: 0.00201937
Epoch:10 average loss: 0.00390016 total validation loss: 0.00185319
Training Time : 0.6797744154930114 minutes 
total test loss of ('elu', 'elu', 'sig', 'relu'): [0.0018738776]
total train loss of ('elu', 'elu', 'sig', 'relu'): [0.0018778779]
[array([[0.24000874],
       [0.21680222],
       [0.4017753 ],
       

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.05085815 total validation loss: 0.04058570
Epoch: 2 average loss: 0.04236198 total validation loss: 0.04052924
Epoch: 3 average loss: 0.04157817 total validation loss: 0.04054617
Epoch: 4 average loss: 0.04121626 total validation loss: 0.04057378
Epoch: 5 average loss: 0.04108049 total validation loss: 0.04057416
Epoch: 6 average loss: 0.04096530 total validation loss: 0.04060358
Epoch: 7 average loss: 0.04098881 total validation loss: 0.04054556
Epoch: 8 average loss: 0.04096273 total validation loss: 0.04053054
Epoch: 9 average loss: 0.04096012 total validation loss: 0.04053014
Epoch:10 average loss: 0.04100398 total validation loss: 0.04052488
Training Time : 0.7665692845980326 minutes 
total test loss of ('elu', 'leaky', 'elu', 'tanh'): [0.040882528]
total train loss of ('elu', 'leaky', 'elu', 'tanh'): [0.040860653]
[array([[0.25059238],
       [0.25059238],
       [0.25059238],
     

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.02080503 total validation loss: 0.00241373
Epoch: 2 average loss: 0.00395473 total validation loss: 0.00166301
Epoch: 3 average loss: 0.00348319 total validation loss: 0.00187579
Epoch: 4 average loss: 0.00322873 total validation loss: 0.00157744
Epoch: 5 average loss: 0.00293221 total validation loss: 0.00154725
Epoch: 6 average loss: 0.00280202 total validation loss: 0.00148897
Epoch: 7 average loss: 0.00271436 total validation loss: 0.00160144
Epoch: 8 average loss: 0.00270986 total validation loss: 0.00162568
Epoch: 9 average loss: 0.00266418 total validation loss: 0.00165811
Epoch:10 average loss: 0.00267909 total validation loss: 0.00163506
Training Time : 0.6899953683217367 minutes 
total test loss of ('elu', 'leaky', 'tanh', 'elu'): [0.0016224617]
total train loss of ('elu', 'leaky', 'tanh', 'elu'): [0.0016421366]
[array([[0.2394731 ],
       [0.19373876],
       [0.36821613],
   

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.04835222 total validation loss: 0.04053320
Epoch: 2 average loss: 0.04223874 total validation loss: 0.04060239
Epoch: 3 average loss: 0.04152223 total validation loss: 0.04052468
Epoch: 4 average loss: 0.04116000 total validation loss: 0.04052278
Epoch: 5 average loss: 0.04101605 total validation loss: 0.04055021
Epoch: 6 average loss: 0.04101597 total validation loss: 0.04061808
Epoch: 7 average loss: 0.04100078 total validation loss: 0.04080821
Epoch: 8 average loss: 0.04097497 total validation loss: 0.04055707
Epoch: 9 average loss: 0.04098329 total validation loss: 0.04053501
Epoch:10 average loss: 0.04097710 total validation loss: 0.04054396
Training Time : 0.6750206192334492 minutes 
total test loss of ('elu', 'relu', 'leaky', 'sig'): [0.04089558]
total train loss of ('elu', 'relu', 'leaky', 'sig'): [0.040870506]
[array([[0.2534409],
       [0.2534409],
       [0.2534409],
       ..

...

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.04788290 total validation loss: 0.04055838
Epoch: 2 average loss: 0.04223539 total validation loss: 0.04055971
Epoch: 3 average loss: 0.04150236 total validation loss: 0.04052406
Epoch: 4 average loss: 0.04111167 total validation loss: 0.04053658
Epoch: 5 average loss: 0.04106274 total validation loss: 0.04059111
Epoch: 6 average loss: 0.04098609 total validation loss: 0.04053819
Epoch: 7 average loss: 0.04098618 total validation loss: 0.04059727
Epoch: 8 average loss: 0.04097437 total validation loss: 0.04063366
Epoch: 9 average loss: 0.04099619 total validation loss: 0.04068282
Epoch:10 average loss: 0.04095521 total validation loss: 0.04061085
Training Time : 0.7112357139587402 minutes 
total test loss of ('tanh', 'elu', 'elu', 'sig'): [0.04095219]
total train loss of ('tanh', 'elu', 'elu', 'sig'): [0.040923454]
[array([[0.25813282],
       [0.25813282],
       [0.25813282],
       ...

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01629854 total validation loss: 0.00240336
Epoch: 2 average loss: 0.00443376 total validation loss: 0.00216482
Epoch: 3 average loss: 0.00379152 total validation loss: 0.00216022
Epoch: 4 average loss: 0.00356104 total validation loss: 0.00212482
Epoch: 5 average loss: 0.00346937 total validation loss: 0.00212032
Epoch: 6 average loss: 0.00342512 total validation loss: 0.00201959
Epoch: 7 average loss: 0.00340325 total validation loss: 0.00231187
Epoch: 8 average loss: 0.00337990 total validation loss: 0.00203183
Epoch: 9 average loss: 0.00334962 total validation loss: 0.00211181
Epoch:10 average loss: 0.00333626 total validation loss: 0.00202790
Training Time : 0.7071497837702433 minutes 
total test loss of ('tanh', 'elu', 'sig', 'elu'): [0.0020422507]
total train loss of ('tanh', 'elu', 'sig', 'elu'): [0.0020515935]
[array([[0.22401384],
       [0.18014291],
       [0.38977784],
       

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.04828833 total validation loss: 0.04052953
Epoch: 2 average loss: 0.04225429 total validation loss: 0.04060409
Epoch: 3 average loss: 0.04154197 total validation loss: 0.04052871
Epoch: 4 average loss: 0.04112554 total validation loss: 0.04055192
Epoch: 5 average loss: 0.04101591 total validation loss: 0.04059364
Epoch: 6 average loss: 0.04101852 total validation loss: 0.04052239
Epoch: 7 average loss: 0.04098469 total validation loss: 0.04053064
Epoch: 8 average loss: 0.04096618 total validation loss: 0.04055239
Epoch: 9 average loss: 0.04098069 total validation loss: 0.04068406
Epoch:10 average loss: 0.04097226 total validation loss: 0.04052675
Training Time : 0.7546011964480083 minutes 
total test loss of ('tanh', 'leaky', 'relu', 'sig'): [0.04088352]
total train loss of ('tanh', 'leaky', 'relu', 'sig'): [0.040861316]
[array([[0.25103587],
       [0.25103587],
       [0.25103587],
    

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.04806541 total validation loss: 0.04052206
Epoch: 2 average loss: 0.04222113 total validation loss: 0.04058499
Epoch: 3 average loss: 0.04144113 total validation loss: 0.04055974
Epoch: 4 average loss: 0.04118688 total validation loss: 0.04056663
Epoch: 5 average loss: 0.04100621 total validation loss: 0.04083660
Epoch: 6 average loss: 0.04100905 total validation loss: 0.04079927
Epoch: 7 average loss: 0.04099613 total validation loss: 0.04052319
Epoch: 8 average loss: 0.04098097 total validation loss: 0.04062904
Epoch: 9 average loss: 0.04099975 total validation loss: 0.04064372
Epoch:10 average loss: 0.04098529 total validation loss: 0.04060836
Training Time : 0.6758864839871724 minutes 
total test loss of ('tanh', 'relu', 'leaky', 'sig'): [0.040949903]
total train loss of ('tanh', 'relu', 'leaky', 'sig'): [0.040921137]
[array([[0.25799975],
       [0.25799975],
       [0.25799975],
   

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.02274422 total validation loss: 0.00248114
Epoch: 2 average loss: 0.00469797 total validation loss: 0.00193054
Epoch: 3 average loss: 0.00389107 total validation loss: 0.00191797
Epoch: 4 average loss: 0.00371660 total validation loss: 0.00184077
Epoch: 5 average loss: 0.00359922 total validation loss: 0.00177305
Epoch: 6 average loss: 0.00355133 total validation loss: 0.00178505
Epoch: 7 average loss: 0.00354835 total validation loss: 0.00177997
Epoch: 8 average loss: 0.00353625 total validation loss: 0.00186807
Epoch: 9 average loss: 0.00352917 total validation loss: 0.00188575
Epoch:10 average loss: 0.00352516 total validation loss: 0.00174658
Training Time : 0.6709131717681884 minutes 
total test loss of ('tanh', 'sig', 'leaky', 'relu'): [0.0017457274]
total train loss of ('tanh', 'sig', 'leaky', 'relu'): [0.0017580691]
[array([[0.26087153],
       [0.20679395],
       [0.37698722],
 

In [11]:
    set_of_activations = ['elu','leaky','relu']
    #set_of_layers = [j for j in set(list(itertools.product(set_of_activations,repeat = num_Layers)))]
    set_of_layers = [j for j in set(list(itertools.product(set_of_activations,repeat = 6)))]
    set_of_layers = sorted(set_of_layers)
    len(set_of_layers)

729

In [18]:
def buildNN_small_size(training_size,numLayers, typeLayers, numNeurons, learning_rate, epoch, batch_size, display_step, dropout_rate, X_train,y_train,X_test,y_test,X_val, y_val, X_data_H, y_data_H,Layer_index=1,model='GBM'):
    #typeLayers is a list of type of layers, numNeurons is a list of # of neurons in each layer
     
    tf.reset_default_graph()
    start_time = time.time()
    iterations = int(X_train.shape[0]/batch_size)
    X_index = np.arange(X_train.shape[0])
    n_input = X_train.shape[1]
    n_classes = 1
    X = tf.placeholder('float', [None, n_input],name='X')
    Y = tf.placeholder('float', [None, n_classes],name='Y')
    drop_rate = tf.placeholder('float',name='drop_rate')
     
    # Store layers weight & bias
    h = []
    b = []
    weightslist = [tf.Variable(tf.random_normal([n_input, numNeurons[0]], stddev= 1/(X_train.shape[0])))]
    biaslist = []
    for i in range(numLayers):
        h.append('h' + str(i + 1))
        b.append('b' + str(i + 1))
        biaslist.append(tf.Variable(tf.random_normal([numNeurons[i]], stddev= 1/(X_train.shape[0]))))
        if i > 0:
            weightslist.append(tf.Variable(tf.random_normal([numNeurons[i - 1], numNeurons[i]], stddev= 1/(X_train.shape[0]))))
    b.append('out')
    h.append('out')
    weightslist.append(tf.Variable(tf.random_normal([numNeurons[numLayers - 1], n_classes], stddev= 1/(X_train.shape[0]))))                          
    biaslist.append(tf.Variable(tf.random_normal([n_classes], stddev= 1/(X_train.shape[0]))))                         
    hcom = zip(h, weightslist)
    weights = dict(hcom)
    bcom = zip(b, biaslist)
    biases = dict(bcom)
     
    a = tf.add(tf.matmul(X, weights['h1']), biases['b1'])
    layer = buildLayer(a, typeLayers[0])
    drop = tf.nn.dropout(layer, keep_prob=1-drop_rate)
    for i in range(1,numLayers):
        a = tf.add(tf.matmul(drop, weights[h[i]]), biases[b[i]])
        layer = buildLayer(a, typeLayers[i])
        drop = tf.nn.dropout(layer, keep_prob=1-drop_rate)
    output_a = tf.add(tf.matmul(drop, weights['out']), biases['out'])  
    logits = tf.exp(output_a, name='logits')
    
    loss = tf.reduce_mean(tf.square(tf.subtract(logits, Y)))
     
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    #optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss)
   
    #Saving the trained model:    
    saver = tf.train.Saver()
    
    init = tf.global_variables_initializer()
   
    with tf.Session() as sess:
        sess.run(init) 
        print("Train on {} samples,".format(len(y_train)),"validate on {} samples,".format(len(y_val)),"test on {} samples".format(len(y_test)))
        for epc in range(epoch):
            np.random.shuffle(X_index)
            avg_loss = 0.0
            for itr in range(iterations):
                sub_index = X_index[itr * batch_size: (1 + itr) * batch_size - 1]
                x_batch = X_train[sub_index]
                y_batch = y_train[sub_index]
                _, loss_val = sess.run([train_op,loss], feed_dict= {X:x_batch, Y:y_batch, drop_rate: dropout_rate})
                avg_loss += loss_val/iterations
            sub_val_loss = sess.run(loss, feed_dict={X:X_val, Y:y_val, drop_rate:0})
            print("Epoch:{:2d}".format(epc+1), "average loss:", "{0:.8f}".format(avg_loss), "total validation loss:", "{0:.8f}".format(sub_val_loss))
        
        #Save the final model:
        meta = './model_{}_{}_'.format(model,training_size)+'{}layers_{}neurons_result_{}'.format(numLayers,numNeurons[0],Layer_index)
        #saver.save(sess,meta,global_step=1000)
        training_time = (((time.time() - start_time))/60)
        saver.save(sess,meta)
        
        
        print("Training Time : %s minutes " % training_time)  
        test_loss = sess.run([loss], feed_dict = {X:X_test, Y:y_test, drop_rate:0})
        print("total test loss of {}: {}".format(typeLayers,test_loss))
        train_loss = sess.run([loss],feed_dict = {X:X_data_H, Y:y_data_H, drop_rate:0})
        print("total train loss of {}: {}".format(typeLayers,train_loss))
        y_train_pred = sess.run([logits],feed_dict={X:X_data_H, Y:y_data_H, drop_rate:0})
        start=time.time()
        y_test_pred = sess.run([logits],feed_dict={X:X_test, Y:y_test, drop_rate:0})
        print(y_test_pred[0:5])
        predict_time = (((time.time() - start)*1000000/len(y_test)))
        print("Average prediction time: {}".format(predict_time))
        W, B = sess.run([weights, biases])
     

    return training_time

In [19]:
def training_small_size(num_Layers,num_Neurons,model,learning_rate = 0.001,epoch =10,batch_size = 64,display_step=1,dropout_rate=0.25,training_size = 40000):
    #Data loading:
    X_train=np.loadtxt('X_train_{}_{}'.format(training_size,model))
    y_train=np.loadtxt('y_train_{}_{}'.format(training_size,model))
    y_train=y_train.reshape(len(y_train),1)
    X_test=np.loadtxt('X_test_{}'.format(model))
    y_test=np.loadtxt('y_test_{}'.format(model))
    y_test=y_test.reshape(len(y_test),1)      
    X_val=np.loadtxt('X_val_{}_{}'.format(training_size,model))
    y_val=np.loadtxt('y_val_{}_{}'.format(training_size,model))
    y_val=y_val.reshape(len(y_val),1)
    X_data_H=np.loadtxt('X_data_H_{}_{}'.format(training_size,model))
    y_data_H=np.loadtxt('y_data_H_{}_{}'.format(training_size,model))
    y_data_H=y_data_H.reshape(len(y_data_H),1)
    
    #Generation of different activation function series:
    set_of_activations = ['elu','leaky','relu']
    set_of_layers = [j for j in set(list(itertools.product(set_of_activations,repeat = num_Layers)))]
    set_of_layers = sorted(set_of_layers)
    #set_of_layers = [['elu','elu','elu','elu','elu','elu'],['leaky','leaky','leaky','leaky','leaky','leaky'],['relu','relu','relu','relu','relu','relu']]
    training_time = []
    #Number of neurons set:
    Neurons=np.ones((num_Layers,), dtype=int)*num_Neurons
    for i in range(len(set_of_layers)):
        print("**********************Training {} - {}:".format(i+1,set_of_layers[i]))
        train_time = buildNN_small_size(training_size,num_Layers, set_of_layers[i], Neurons, learning_rate, epoch, batch_size, display_step, dropout_rate, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,(i+1),model)
        training_time.append(train_time) 
    np.savetxt('training_time_{}_{}_'.format(training_size,model)+'{}layers_{}neurons'.format(num_Layers,num_Neurons),training_time)
    return set_of_layers,training_time

In [21]:
#40000 -- 4 layers & 120 neurons & 'GBM' :
set_of_layers,training_time = training_small_size(4,120,'GBM',0.001,10,64,1,0.25,40000)
#80000 -- 4 layers & 120 neurons & 'GBM' :
set_of_layers,training_time = training_small_size(4,120,'GBM',0.001,10,64,1,0.25,80000)
#160000 -- 4 layers & 120 neurons & 'GBM' :
set_of_layers,training_time = training_small_size(4,120,'GBM',0.001,10,64,1,0.25,160000)

**********************Training 1 - ('elu', 'elu', 'elu', 'elu'):
Train on 36000 samples, validate on 4000 samples, test on 60000 samples
Epoch: 1 average loss: 0.03552345 total validation loss: 0.00283571
Epoch: 2 average loss: 0.00329409 total validation loss: 0.00230472
Epoch: 3 average loss: 0.00284294 total validation loss: 0.00200697
Epoch: 4 average loss: 0.00268989 total validation loss: 0.00184557
Epoch: 5 average loss: 0.00246259 total validation loss: 0.00132101
Epoch: 6 average loss: 0.00200228 total validation loss: 0.00123000
Epoch: 7 average loss: 0.00188745 total validation loss: 0.00119521
Epoch: 8 average loss: 0.00178641 total validation loss: 0.00124911
Epoch: 9 average loss: 0.00175170 total validation loss: 0.00129710
Epoch:10 average loss: 0.00173078 total validation loss: 0.00124269
Training Time : 0.1946986516316732 minutes 
total test loss of ('elu', 'elu', 'elu', 'elu'): [0.001202641]
total train loss of ('elu', 'elu', 'elu', 'elu'): [0.0012047281]
[array([[0.

Train on 36000 samples, validate on 4000 samples, test on 60000 samples
Epoch: 1 average loss: 0.04153653 total validation loss: 0.00190356
Epoch: 2 average loss: 0.00250722 total validation loss: 0.00125071
Epoch: 3 average loss: 0.00217340 total validation loss: 0.00127699
Epoch: 4 average loss: 0.00195012 total validation loss: 0.00116917
Epoch: 5 average loss: 0.00174444 total validation loss: 0.00104087
Epoch: 6 average loss: 0.00129045 total validation loss: 0.00042581
Epoch: 7 average loss: 0.00091453 total validation loss: 0.00020712
Epoch: 8 average loss: 0.00085518 total validation loss: 0.00018751
Epoch: 9 average loss: 0.00079016 total validation loss: 0.00022150
Epoch:10 average loss: 0.00074518 total validation loss: 0.00017555
Training Time : 0.20791614850362142 minutes 
total test loss of ('elu', 'elu', 'relu', 'leaky'): [0.000183402]
total train loss of ('elu', 'elu', 'relu', 'leaky'): [0.00018202762]
[array([[0.33933097],
       [0.46472916],
       [0.47164142],
    

Train on 36000 samples, validate on 4000 samples, test on 60000 samples
Epoch: 1 average loss: 0.04486370 total validation loss: 0.00133321
Epoch: 2 average loss: 0.00257804 total validation loss: 0.00113380
Epoch: 3 average loss: 0.00233943 total validation loss: 0.00105357
Epoch: 4 average loss: 0.00213419 total validation loss: 0.00054719
Epoch: 5 average loss: 0.00158204 total validation loss: 0.00037437
Epoch: 6 average loss: 0.00143423 total validation loss: 0.00026396
Epoch: 7 average loss: 0.00135742 total validation loss: 0.00022701
Epoch: 8 average loss: 0.00123591 total validation loss: 0.00026389
Epoch: 9 average loss: 0.00113839 total validation loss: 0.00025613
Epoch:10 average loss: 0.00105737 total validation loss: 0.00020242
Training Time : 0.21220374902089437 minutes 
total test loss of ('elu', 'leaky', 'leaky', 'relu'): [0.00020642331]
total train loss of ('elu', 'leaky', 'leaky', 'relu'): [0.00020693021]
[array([[0.33618075],
       [0.47193217],
       [0.47125268]

Train on 36000 samples, validate on 4000 samples, test on 60000 samples
Epoch: 1 average loss: 0.03843077 total validation loss: 0.00120753
Epoch: 2 average loss: 0.00187599 total validation loss: 0.00100909
Epoch: 3 average loss: 0.00159249 total validation loss: 0.00041891
Epoch: 4 average loss: 0.00099982 total validation loss: 0.00021436
Epoch: 5 average loss: 0.00081951 total validation loss: 0.00051539
Epoch: 6 average loss: 0.00077220 total validation loss: 0.00014267
Epoch: 7 average loss: 0.00071664 total validation loss: 0.00016917
Epoch: 8 average loss: 0.00069699 total validation loss: 0.00013288
Epoch: 9 average loss: 0.00066958 total validation loss: 0.00020886
Epoch:10 average loss: 0.00065295 total validation loss: 0.00022373
Training Time : 0.2378483017285665 minutes 
total test loss of ('elu', 'relu', 'leaky', 'elu'): [0.0002348477]
total train loss of ('elu', 'relu', 'leaky', 'elu'): [0.000232305]
[array([[0.3540123 ],
       [0.5048101 ],
       [0.49881485],
      

Train on 36000 samples, validate on 4000 samples, test on 60000 samples
Epoch: 1 average loss: 0.03767345 total validation loss: 0.00097103
Epoch: 2 average loss: 0.00169745 total validation loss: 0.00044085
Epoch: 3 average loss: 0.00110289 total validation loss: 0.00033301
Epoch: 4 average loss: 0.00100493 total validation loss: 0.00036448
Epoch: 5 average loss: 0.00092099 total validation loss: 0.00025439
Epoch: 6 average loss: 0.00087718 total validation loss: 0.00027358
Epoch: 7 average loss: 0.00088113 total validation loss: 0.00023641
Epoch: 8 average loss: 0.00084391 total validation loss: 0.00026579
Epoch: 9 average loss: 0.00086142 total validation loss: 0.00023091
Epoch:10 average loss: 0.00079001 total validation loss: 0.00023294
Training Time : 0.24316275119781494 minutes 
total test loss of ('leaky', 'elu', 'elu', 'leaky'): [0.00024307724]
total train loss of ('leaky', 'elu', 'elu', 'leaky'): [0.00024226199]
[array([[0.3489332 ],
       [0.5005605 ],
       [0.4883108 ],


...

Epoch:10 average loss: 0.00033328 total validation loss: 0.00024974
Training Time : 91.8212090174357 minutes 
total test loss of ('leaky', 'relu', 'relu', 'leaky'): [0.00026298742]
total train loss of ('leaky', 'relu', 'relu', 'leaky'): [0.00026022535]
[array([[0.32524592],
       [0.45665783],
       [0.45712665],
       ...,
       [0.3439551 ],
       [0.13151652],
       [0.10658251]], dtype=float32)]
Average prediction time: 5.620849132537842
**********************Training 54 - ('leaky', 'relu', 'relu', 'relu'):
Train on 144000 samples, validate on 16000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01431953 total validation loss: 0.00021864
Epoch: 2 average loss: 0.00114161 total validation loss: 0.00011965
Epoch: 3 average loss: 0.00094294 total validation loss: 0.00007699
Epoch: 4 average loss: 0.00071921 total validation loss: 0.00010176
Epoch: 5 average loss: 0.00054495 total validation loss: 0.00008636
Epoch: 6 average loss: 0.00052037 total validation loss: 0.0000

Epoch: 7 average loss: 0.00053470 total validation loss: 0.00013911
Epoch: 8 average loss: 0.00052722 total validation loss: 0.00013500
Epoch: 9 average loss: 0.00051280 total validation loss: 0.00011712
Epoch:10 average loss: 0.00050787 total validation loss: 0.00012175
Training Time : 1.832371453444163 minutes 
total test loss of ('relu', 'elu', 'leaky', 'relu'): [0.00012718789]
total train loss of ('relu', 'elu', 'leaky', 'relu'): [0.00012543361]
[array([[0.33943936],
       [0.47872344],
       [0.4779931 ],
       ...,
       [0.36441147],
       [0.13000707],
       [0.10310583]], dtype=float32)]
Average prediction time: 5.628033479054769
**********************Training 61 - ('relu', 'elu', 'relu', 'elu'):
Train on 144000 samples, validate on 16000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01068286 total validation loss: 0.00037309
Epoch: 2 average loss: 0.00088325 total validation loss: 0.00021455
Epoch: 3 average loss: 0.00078895 total validation loss: 0.00052817
E

Epoch: 4 average loss: 0.00062778 total validation loss: 0.00011342
Epoch: 5 average loss: 0.00053749 total validation loss: 0.00043574
Epoch: 6 average loss: 0.00046646 total validation loss: 0.00021157
Epoch: 7 average loss: 0.00041779 total validation loss: 0.00011950
Epoch: 8 average loss: 0.00036351 total validation loss: 0.00021176
Epoch: 9 average loss: 0.00033858 total validation loss: 0.00013699
Epoch:10 average loss: 0.00032302 total validation loss: 0.00012304
Training Time : 0.9316255648930868 minutes 
total test loss of ('relu', 'leaky', 'leaky', 'elu'): [0.00012753083]
total train loss of ('relu', 'leaky', 'leaky', 'elu'): [0.0001270159]
[array([[0.3424075 ],
       [0.4680922 ],
       [0.4751679 ],
       ...,
       [0.36722088],
       [0.13395382],
       [0.11020967]], dtype=float32)]
Average prediction time: 5.684951941172282
**********************Training 68 - ('relu', 'leaky', 'leaky', 'leaky'):
Train on 144000 samples, validate on 16000 samples, test on 60000 sa

Epoch: 1 average loss: 0.01120628 total validation loss: 0.00022571
Epoch: 2 average loss: 0.00096973 total validation loss: 0.00025363
Epoch: 3 average loss: 0.00081481 total validation loss: 0.00018366
Epoch: 4 average loss: 0.00072551 total validation loss: 0.00014660
Epoch: 5 average loss: 0.00064107 total validation loss: 0.00011296
Epoch: 6 average loss: 0.00056526 total validation loss: 0.00032620
Epoch: 7 average loss: 0.00053037 total validation loss: 0.00012929
Epoch: 8 average loss: 0.00049746 total validation loss: 0.00012157
Epoch: 9 average loss: 0.00047604 total validation loss: 0.00016210
Epoch:10 average loss: 0.00044809 total validation loss: 0.00010193
Training Time : 0.8647617141405741 minutes 
total test loss of ('relu', 'relu', 'elu', 'leaky'): [0.00010414971]
total train loss of ('relu', 'relu', 'elu', 'leaky'): [0.000103981474]
[array([[0.34634686],
       [0.47787893],
       [0.48023027],
       ...,
       [0.3793859 ],
       [0.1306809 ],
       [0.1032684 

Train on 144000 samples, validate on 16000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01427331 total validation loss: 0.00019067
Epoch: 2 average loss: 0.00129915 total validation loss: 0.00023953
Epoch: 3 average loss: 0.00108157 total validation loss: 0.00019994
Epoch: 4 average loss: 0.00085065 total validation loss: 0.00013445
Epoch: 5 average loss: 0.00066063 total validation loss: 0.00014312
Epoch: 6 average loss: 0.00059670 total validation loss: 0.00018563
Epoch: 7 average loss: 0.00058162 total validation loss: 0.00013920
Epoch: 8 average loss: 0.00056906 total validation loss: 0.00017498
Epoch: 9 average loss: 0.00054237 total validation loss: 0.00016418
Epoch:10 average loss: 0.00053765 total validation loss: 0.00021617
Training Time : 0.7052583336830139 minutes 
total test loss of ('relu', 'relu', 'relu', 'relu'): [0.00022015642]
total train loss of ('relu', 'relu', 'relu', 'relu'): [0.00021903063]
[array([[0.36153597],
       [0.49971202],
       [0.49534693],

In [22]:
#40000 -- 2 layers & 120 neurons & 'GBM' :
set_of_layers,training_time = training_small_size(2,120,'GBM',0.001,10,64,1,0.25,40000)
#80000 -- 2 layers & 120 neurons & 'GBM' :
set_of_layers,training_time = training_small_size(2,120,'GBM',0.001,10,64,1,0.25,80000)
#160000 -- 2 layers & 120 neurons & 'GBM' :
set_of_layers,training_time = training_small_size(2,120,'GBM',0.001,10,64,1,0.25,160000)

**********************Training 1 - ('elu', 'elu'):
Train on 36000 samples, validate on 4000 samples, test on 60000 samples
Epoch: 1 average loss: 0.04064806 total validation loss: 0.00624123
Epoch: 2 average loss: 0.00344374 total validation loss: 0.00201248
Epoch: 3 average loss: 0.00257175 total validation loss: 0.00194961
Epoch: 4 average loss: 0.00247560 total validation loss: 0.00202263
Epoch: 5 average loss: 0.00243540 total validation loss: 0.00191389
Epoch: 6 average loss: 0.00233361 total validation loss: 0.00185995
Epoch: 7 average loss: 0.00227766 total validation loss: 0.00183827
Epoch: 8 average loss: 0.00228741 total validation loss: 0.00201623
Epoch: 9 average loss: 0.00227252 total validation loss: 0.00178930
Epoch:10 average loss: 0.00222669 total validation loss: 0.00190537
Training Time : 0.1251147508621216 minutes 
total test loss of ('elu', 'elu'): [0.0019003334]
total train loss of ('elu', 'elu'): [0.0019045776]
[array([[0.2847334 ],
       [0.48693395],
       [0

Epoch: 1 average loss: 0.04845396 total validation loss: 0.00792219
Epoch: 2 average loss: 0.00481316 total validation loss: 0.00169413
Epoch: 3 average loss: 0.00160338 total validation loss: 0.00061903
Epoch: 4 average loss: 0.00107824 total validation loss: 0.00045000
Epoch: 5 average loss: 0.00093045 total validation loss: 0.00030507
Epoch: 6 average loss: 0.00085089 total validation loss: 0.00026892
Epoch: 7 average loss: 0.00082768 total validation loss: 0.00024081
Epoch: 8 average loss: 0.00078122 total validation loss: 0.00020421
Epoch: 9 average loss: 0.00074942 total validation loss: 0.00016618
Epoch:10 average loss: 0.00074656 total validation loss: 0.00018843
Training Time : 0.13252053260803223 minutes 
total test loss of ('relu', 'leaky'): [0.00020999114]
total train loss of ('relu', 'leaky'): [0.00020394723]
[array([[0.34332013],
       [0.50125605],
       [0.4904536 ],
       ...,
       [0.37236384],
       [0.12250611],
       [0.09737988]], dtype=float32)]
Average pr

Epoch: 3 average loss: 0.00099179 total validation loss: 0.00025545
Epoch: 4 average loss: 0.00089682 total validation loss: 0.00023245
Epoch: 5 average loss: 0.00083448 total validation loss: 0.00016025
Epoch: 6 average loss: 0.00074369 total validation loss: 0.00014939
Epoch: 7 average loss: 0.00063900 total validation loss: 0.00013679
Epoch: 8 average loss: 0.00052014 total validation loss: 0.00010192
Epoch: 9 average loss: 0.00043887 total validation loss: 0.00009439
Epoch:10 average loss: 0.00041443 total validation loss: 0.00009852
Training Time : 0.27180454730987547 minutes 
total test loss of ('leaky', 'relu'): [9.776905e-05]
total train loss of ('leaky', 'relu'): [9.721006e-05]
[array([[0.34059286],
       [0.48396215],
       [0.48231852],
       ...,
       [0.36109367],
       [0.12950984],
       [0.09889188]], dtype=float32)]
Average prediction time: 2.980184555053711
**********************Training 7 - ('relu', 'elu'):
Train on 72000 samples, validate on 8000 samples, tes

Epoch: 5 average loss: 0.00057032 total validation loss: 0.00021563
Epoch: 6 average loss: 0.00053085 total validation loss: 0.00023478
Epoch: 7 average loss: 0.00049716 total validation loss: 0.00020841
Epoch: 8 average loss: 0.00047487 total validation loss: 0.00020819
Epoch: 9 average loss: 0.00046202 total validation loss: 0.00024229
Epoch:10 average loss: 0.00045403 total validation loss: 0.00018993
Training Time : 0.5091949820518493 minutes 
total test loss of ('leaky', 'elu'): [0.00019706393]
total train loss of ('leaky', 'elu'): [0.00019895921]
[array([[0.34448463],
       [0.48256898],
       [0.47564155],
       ...,
       [0.3634645 ],
       [0.12674163],
       [0.09781013]], dtype=float32)]
Average prediction time: 3.142698605855306
**********************Training 5 - ('leaky', 'leaky'):
Train on 144000 samples, validate on 16000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01360323 total validation loss: 0.00028788
Epoch: 2 average loss: 0.00070242 total valid

In [11]:
set_of_layers,training_time = training_small_size(2,120,'JuZhong',0.001,10,64,1,0.25,80000)

**********************Training 1 - ('elu', 'elu'):
Train on 72000 samples, validate on 8000 samples, test on 8000 samples
Epoch: 1 average loss: 0.02230252 total validation loss: 0.00241103
Epoch: 2 average loss: 0.00247957 total validation loss: 0.00227427
Epoch: 3 average loss: 0.00233676 total validation loss: 0.00194681
Epoch: 4 average loss: 0.00228247 total validation loss: 0.00183137
Epoch: 5 average loss: 0.00223562 total validation loss: 0.00192506
Epoch: 6 average loss: 0.00211154 total validation loss: 0.00167540
Epoch: 7 average loss: 0.00188342 total validation loss: 0.00127959
Epoch: 8 average loss: 0.00168044 total validation loss: 0.00120135
Epoch: 9 average loss: 0.00154887 total validation loss: 0.00116713
Epoch:10 average loss: 0.00144685 total validation loss: 0.00111225
Training Time : 0.2489775816599528 minutes 
total test loss of ('elu', 'elu'): [0.0011122461]
total train loss of ('elu', 'elu'): [0.0011061949]
[array([[0.10344173],
       [0.08114249],
       [0.

Epoch: 1 average loss: 0.02803545 total validation loss: 0.00189827
Epoch: 2 average loss: 0.00143956 total validation loss: 0.00041696
Epoch: 3 average loss: 0.00092123 total validation loss: 0.00028520
Epoch: 4 average loss: 0.00081328 total validation loss: 0.00024931
Epoch: 5 average loss: 0.00076174 total validation loss: 0.00022030
Epoch: 6 average loss: 0.00071908 total validation loss: 0.00020487
Epoch: 7 average loss: 0.00069150 total validation loss: 0.00019682
Epoch: 8 average loss: 0.00065910 total validation loss: 0.00017857
Epoch: 9 average loss: 0.00061989 total validation loss: 0.00016538
Epoch:10 average loss: 0.00058437 total validation loss: 0.00027861
Training Time : 0.3012591004371643 minutes 
total test loss of ('relu', 'leaky'): [0.0002786129]
total train loss of ('relu', 'leaky'): [0.0002664335]
[array([[0.09224317],
       [0.05310748],
       [0.26556093],
       ...,
       [0.05891108],
       [0.13999537],
       [0.08995761]], dtype=float32)]
Average predi

In [22]:
#40000 -- 2 layers & 120 neurons & 'BarrierDownIn' :
set_of_layers,training_time = training_small_size(2,120,'BarrierDownIn',0.001,10,64,1,0.25,40000)
#80000 -- 2 layers & 120 neurons & 'BarrierDownIn' :
set_of_layers,training_time = training_small_size(2,120,'BarrierDownIn',0.001,10,64,1,0.25,80000)
#160000 -- 2 layers & 120 neurons & 'BarrierDownIn' :
set_of_layers,training_time = training_small_size(2,120,'BarrierDownIn',0.001,10,64,1,0.25,160000)

**********************Training 1 - ('elu', 'elu'):
Train on 36000 samples, validate on 4000 samples, test on 4000 samples
Epoch: 1 average loss: 0.04314856 total validation loss: 0.00669017
Epoch: 2 average loss: 0.00545301 total validation loss: 0.00330837
Epoch: 3 average loss: 0.00261638 total validation loss: 0.00173051
Epoch: 4 average loss: 0.00222790 total validation loss: 0.00162205
Epoch: 5 average loss: 0.00209463 total validation loss: 0.00171006
Epoch: 6 average loss: 0.00203181 total validation loss: 0.00156948
Epoch: 7 average loss: 0.00196669 total validation loss: 0.00154322
Epoch: 8 average loss: 0.00197857 total validation loss: 0.00161112
Epoch: 9 average loss: 0.00197023 total validation loss: 0.00150049
Epoch:10 average loss: 0.00192547 total validation loss: 0.00145184
Training Time : 0.13479692935943605 minutes 
total test loss of ('elu', 'elu'): [0.0014518405]
total train loss of ('elu', 'elu'): [0.0014917173]
[array([[0.15199101],
       [0.32188064],
       [0

Epoch: 1 average loss: 0.05267968 total validation loss: 0.00801460
Epoch: 2 average loss: 0.00619163 total validation loss: 0.00438910
Epoch: 3 average loss: 0.00350758 total validation loss: 0.00186290
Epoch: 4 average loss: 0.00155795 total validation loss: 0.00072044
Epoch: 5 average loss: 0.00091324 total validation loss: 0.00035219
Epoch: 6 average loss: 0.00072573 total validation loss: 0.00024148
Epoch: 7 average loss: 0.00064131 total validation loss: 0.00018522
Epoch: 8 average loss: 0.00063330 total validation loss: 0.00027894
Epoch: 9 average loss: 0.00060247 total validation loss: 0.00017633
Epoch:10 average loss: 0.00055616 total validation loss: 0.00019443
Training Time : 0.2030454476674398 minutes 
total test loss of ('relu', 'leaky'): [0.00019443262]
total train loss of ('relu', 'leaky'): [0.0002099721]
[array([[0.16109565],
       [0.36185288],
       [0.02649881],
       ...,
       [0.03398092],
       [0.2589238 ],
       [0.02067336]], dtype=float32)]
Average pred

Epoch: 3 average loss: 0.00082121 total validation loss: 0.00017909
Epoch: 4 average loss: 0.00067526 total validation loss: 0.00013618
Epoch: 5 average loss: 0.00060383 total validation loss: 0.00021814
Epoch: 6 average loss: 0.00053817 total validation loss: 0.00008154
Epoch: 7 average loss: 0.00048632 total validation loss: 0.00006534
Epoch: 8 average loss: 0.00042838 total validation loss: 0.00005099
Epoch: 9 average loss: 0.00036444 total validation loss: 0.00004295
Epoch:10 average loss: 0.00029705 total validation loss: 0.00004656
Training Time : 0.28779462973276776 minutes 
total test loss of ('leaky', 'relu'): [4.655642e-05]
total train loss of ('leaky', 'relu'): [5.002611e-05]
[array([[0.22414012],
       [0.24852166],
       [0.3003703 ],
       ...,
       [0.00376127],
       [0.13359421],
       [0.0624645 ]], dtype=float32)]
Average prediction time: 3.2335221767425537
**********************Training 7 - ('relu', 'elu'):
Train on 72000 samples, validate on 8000 samples, te

Epoch: 5 average loss: 0.00043327 total validation loss: 0.00023768
Epoch: 6 average loss: 0.00039985 total validation loss: 0.00023452
Epoch: 7 average loss: 0.00037325 total validation loss: 0.00015299
Epoch: 8 average loss: 0.00035148 total validation loss: 0.00014419
Epoch: 9 average loss: 0.00033918 total validation loss: 0.00017782
Epoch:10 average loss: 0.00032813 total validation loss: 0.00014847
Training Time : 0.5411370158195495 minutes 
total test loss of ('leaky', 'elu'): [0.00014846511]
total train loss of ('leaky', 'elu'): [0.00014513101]
[array([[0.43074757],
       [0.2876562 ],
       [0.06942271],
       ...,
       [0.22924192],
       [0.11835121],
       [0.01129595]], dtype=float32)]
Average prediction time: 3.6478042602539062
**********************Training 5 - ('leaky', 'leaky'):
Train on 144000 samples, validate on 16000 samples, test on 16000 samples
Epoch: 1 average loss: 0.01554824 total validation loss: 0.00068040
Epoch: 2 average loss: 0.00067532 total vali

In [23]:
#40000 -- 4 layers & 120 neurons & 'BarrierDownIn' :
set_of_layers,training_time = training_small_size(4,120,'BarrierDownIn',0.001,10,64,1,0.25,40000)
#80000 -- 4 layers & 120 neurons & 'BarrierDownIn' :
set_of_layers,training_time = training_small_size(4,120,'BarrierDownIn',0.001,10,64,1,0.25,80000)
#160000 -- 4 layers & 120 neurons & 'BarrierDownIn' :
#Data load:
model = 'GBM'
X_train=np.loadtxt('X_train_%s'% model)
y_train=np.loadtxt('y_train_%s'% model)
y_train=y_train.reshape(len(y_train),1)
X_test=np.loadtxt('X_test_%s'% model)
y_test=np.loadtxt('y_test_%s'% model)
y_test=y_test.reshape(len(y_test),1)
X_val=np.loadtxt('X_val_%s'% model)
y_val=np.loadtxt('y_val_%s'% model)
y_val=y_val.reshape(len(y_val),1)
X_data_H=np.loadtxt('X_data_H_%s'% model)
y_data_H=np.loadtxt('y_data_H_%s'% model)
y_data_H=y_data_H.reshape(len(y_data_H),1)set_of_layers,training_time = training_small_size(4,120,'BarrierDownIn',0.001,10,64,1,0.25,160000)

**********************Training 1 - ('elu', 'elu', 'elu', 'elu'):
Train on 36000 samples, validate on 4000 samples, test on 4000 samples
Epoch: 1 average loss: 0.05525803 total validation loss: 0.01004648
Epoch: 2 average loss: 0.00493119 total validation loss: 0.00217834
Epoch: 3 average loss: 0.00287309 total validation loss: 0.00216885
Epoch: 4 average loss: 0.00245654 total validation loss: 0.00154840
Epoch: 5 average loss: 0.00235886 total validation loss: 0.00149797
Epoch: 6 average loss: 0.00197757 total validation loss: 0.00102438
Epoch: 7 average loss: 0.00150005 total validation loss: 0.00085227
Epoch: 8 average loss: 0.00134023 total validation loss: 0.00081940
Epoch: 9 average loss: 0.00125581 total validation loss: 0.00077272
Epoch:10 average loss: 0.00120923 total validation loss: 0.00104030
Training Time : 0.21723090012868246 minutes 
total test loss of ('elu', 'elu', 'elu', 'elu'): [0.0010402987]
total train loss of ('elu', 'elu', 'elu', 'elu'): [0.0010358851]
[array([[0

Train on 36000 samples, validate on 4000 samples, test on 4000 samples
Epoch: 1 average loss: 0.05062485 total validation loss: 0.00438396
Epoch: 2 average loss: 0.00274362 total validation loss: 0.00086302
Epoch: 3 average loss: 0.00182149 total validation loss: 0.00082592
Epoch: 4 average loss: 0.00159403 total validation loss: 0.00070057
Epoch: 5 average loss: 0.00128086 total validation loss: 0.00067557
Epoch: 6 average loss: 0.00120496 total validation loss: 0.00070194
Epoch: 7 average loss: 0.00110849 total validation loss: 0.00069343
Epoch: 8 average loss: 0.00107339 total validation loss: 0.00031242
Epoch: 9 average loss: 0.00067104 total validation loss: 0.00014460
Epoch:10 average loss: 0.00060055 total validation loss: 0.00013807
Training Time : 0.257219131787618 minutes 
total test loss of ('elu', 'elu', 'relu', 'leaky'): [0.00013806557]
total train loss of ('elu', 'elu', 'relu', 'leaky'): [0.00014681181]
[array([[0.17353292],
       [0.34736112],
       [0.02274668],
     

Train on 36000 samples, validate on 4000 samples, test on 4000 samples
Epoch: 1 average loss: 0.05370147 total validation loss: 0.00368518
Epoch: 2 average loss: 0.00230004 total validation loss: 0.00089234
Epoch: 3 average loss: 0.00146471 total validation loss: 0.00080456
Epoch: 4 average loss: 0.00135587 total validation loss: 0.00074786
Epoch: 5 average loss: 0.00132607 total validation loss: 0.00070947
Epoch: 6 average loss: 0.00114604 total validation loss: 0.00028460
Epoch: 7 average loss: 0.00081913 total validation loss: 0.00020846
Epoch: 8 average loss: 0.00076802 total validation loss: 0.00012547
Epoch: 9 average loss: 0.00073455 total validation loss: 0.00016373
Epoch:10 average loss: 0.00069075 total validation loss: 0.00009667
Training Time : 0.27118874788284303 minutes 
total test loss of ('elu', 'leaky', 'leaky', 'relu'): [9.667224e-05]
total train loss of ('elu', 'leaky', 'leaky', 'relu'): [9.91016e-05]
[array([[0.17823778],
       [0.3609408 ],
       [0.01875506],
  

Train on 36000 samples, validate on 4000 samples, test on 4000 samples
Epoch: 1 average loss: 0.04415145 total validation loss: 0.00251088
Epoch: 2 average loss: 0.00174123 total validation loss: 0.00086543
Epoch: 3 average loss: 0.00124700 total validation loss: 0.00083743
Epoch: 4 average loss: 0.00107678 total validation loss: 0.00030025
Epoch: 5 average loss: 0.00068977 total validation loss: 0.00017919
Epoch: 6 average loss: 0.00058224 total validation loss: 0.00008780
Epoch: 7 average loss: 0.00052321 total validation loss: 0.00012479
Epoch: 8 average loss: 0.00052048 total validation loss: 0.00012053
Epoch: 9 average loss: 0.00048116 total validation loss: 0.00010126
Epoch:10 average loss: 0.00046149 total validation loss: 0.00009446
Training Time : 0.29122423330942787 minutes 
total test loss of ('elu', 'relu', 'leaky', 'elu'): [9.445739e-05]
total train loss of ('elu', 'relu', 'leaky', 'elu'): [9.5727e-05]
[array([[0.16410474],
       [0.36517906],
       [0.02108903],
       

Train on 36000 samples, validate on 4000 samples, test on 4000 samples
Epoch: 1 average loss: 0.06165809 total validation loss: 0.02010971
Epoch: 2 average loss: 0.00430946 total validation loss: 0.00077913
Epoch: 3 average loss: 0.00121621 total validation loss: 0.00062816
Epoch: 4 average loss: 0.00081035 total validation loss: 0.00021819
Epoch: 5 average loss: 0.00071970 total validation loss: 0.00023428
Epoch: 6 average loss: 0.00071774 total validation loss: 0.00015347
Epoch: 7 average loss: 0.00063793 total validation loss: 0.00023113
Epoch: 8 average loss: 0.00065315 total validation loss: 0.00014825
Epoch: 9 average loss: 0.00063156 total validation loss: 0.00021552
Epoch:10 average loss: 0.00059607 total validation loss: 0.00014730
Training Time : 0.31618868509928383 minutes 
total test loss of ('leaky', 'elu', 'elu', 'leaky'): [0.00014729702]
total train loss of ('leaky', 'elu', 'elu', 'leaky'): [0.00015062788]
[array([[0.17887266],
       [0.36081982],
       [0.02508624],
 

...

[array([[0.44873   ],
       [0.30289707],
       [0.07546469],
       ...,
       [0.22938314],
       [0.12317072],
       [0.00445339]], dtype=float32)]
Average prediction time: 7.082000374794006
**********************Training 48 - ('leaky', 'relu', 'elu', 'relu'):
Train on 144000 samples, validate on 16000 samples, test on 16000 samples
Epoch: 1 average loss: 0.01319107 total validation loss: 0.00013277
Epoch: 2 average loss: 0.00072996 total validation loss: 0.00006624
Epoch: 3 average loss: 0.00062610 total validation loss: 0.00009765
Epoch: 4 average loss: 0.00052053 total validation loss: 0.00013932
Epoch: 5 average loss: 0.00043051 total validation loss: 0.00011454
Epoch: 6 average loss: 0.00033864 total validation loss: 0.00013133
Epoch: 7 average loss: 0.00029242 total validation loss: 0.00009905
Epoch: 8 average loss: 0.00027551 total validation loss: 0.00012998
Epoch: 9 average loss: 0.00026803 total validation loss: 0.00005680
Epoch:10 average loss: 0.00025893 total valid

Epoch:10 average loss: 0.00038293 total validation loss: 0.00009415
Training Time : 0.801307487487793 minutes 
total test loss of ('leaky', 'relu', 'relu', 'relu'): [9.415409e-05]
total train loss of ('leaky', 'relu', 'relu', 'relu'): [9.6836135e-05]
[array([[0.39671075],
       [0.28930023],
       [0.06383992],
       ...,
       [0.22002764],
       [0.11636779],
       [0.00336341]], dtype=float32)]
Average prediction time: 6.695374846458435
**********************Training 55 - ('relu', 'elu', 'elu', 'elu'):
Train on 144000 samples, validate on 16000 samples, test on 16000 samples
Epoch: 1 average loss: 0.01288996 total validation loss: 0.00038711
Epoch: 2 average loss: 0.00068299 total validation loss: 0.00008993
Epoch: 3 average loss: 0.00056510 total validation loss: 0.00012982
Epoch: 4 average loss: 0.00050113 total validation loss: 0.00011237
Epoch: 5 average loss: 0.00045050 total validation loss: 0.00010158
Epoch: 6 average loss: 0.00038700 total validation loss: 0.00014848
E

Epoch: 7 average loss: 0.00036412 total validation loss: 0.00010695
Epoch: 8 average loss: 0.00033953 total validation loss: 0.00008626
Epoch: 9 average loss: 0.00031886 total validation loss: 0.00008136
Epoch:10 average loss: 0.00030596 total validation loss: 0.00006185
Training Time : 0.8100451946258544 minutes 
total test loss of ('relu', 'elu', 'relu', 'elu'): [6.185258e-05]
total train loss of ('relu', 'elu', 'relu', 'elu'): [6.534418e-05]
[array([[0.4526293 ],
       [0.29033813],
       [0.07917964],
       ...,
       [0.21983974],
       [0.12023526],
       [0.00540693]], dtype=float32)]
Average prediction time: 7.212191820144653
**********************Training 62 - ('relu', 'elu', 'relu', 'leaky'):
Train on 144000 samples, validate on 16000 samples, test on 16000 samples
Epoch: 1 average loss: 0.01351997 total validation loss: 0.00036850
Epoch: 2 average loss: 0.00073266 total validation loss: 0.00012657
Epoch: 3 average loss: 0.00057982 total validation loss: 0.00015995
Epoc

Epoch: 4 average loss: 0.00051531 total validation loss: 0.00006412
Epoch: 5 average loss: 0.00044953 total validation loss: 0.00006763
Epoch: 6 average loss: 0.00038095 total validation loss: 0.00011098
Epoch: 7 average loss: 0.00034034 total validation loss: 0.00006093
Epoch: 8 average loss: 0.00030538 total validation loss: 0.00007055
Epoch: 9 average loss: 0.00028493 total validation loss: 0.00008776
Epoch:10 average loss: 0.00027863 total validation loss: 0.00005218
Training Time : 0.8590546290079752 minutes 
total test loss of ('relu', 'leaky', 'leaky', 'leaky'): [5.217678e-05]
total train loss of ('relu', 'leaky', 'leaky', 'leaky'): [5.3067266e-05]
[array([[0.4350471 ],
       [0.2855622 ],
       [0.07996756],
       ...,
       [0.22490673],
       [0.12565902],
       [0.00454359]], dtype=float32)]
Average prediction time: 8.127376437187195
**********************Training 69 - ('relu', 'leaky', 'leaky', 'relu'):
Train on 144000 samples, validate on 16000 samples, test on 16000

Epoch: 1 average loss: 0.01403189 total validation loss: 0.00041480
Epoch: 2 average loss: 0.00100394 total validation loss: 0.00040111
Epoch: 3 average loss: 0.00084625 total validation loss: 0.00014035
Epoch: 4 average loss: 0.00073301 total validation loss: 0.00013767
Epoch: 5 average loss: 0.00059782 total validation loss: 0.00011720
Epoch: 6 average loss: 0.00048596 total validation loss: 0.00009495
Epoch: 7 average loss: 0.00042558 total validation loss: 0.00009846
Epoch: 8 average loss: 0.00039758 total validation loss: 0.00008688
Epoch: 9 average loss: 0.00037903 total validation loss: 0.00005136
Epoch:10 average loss: 0.00036549 total validation loss: 0.00013398
Training Time : 0.8021946509679159 minutes 
total test loss of ('relu', 'relu', 'elu', 'relu'): [0.00013398365]
total train loss of ('relu', 'relu', 'elu', 'relu'): [0.00014105957]
[array([[0.39566165],
       [0.28701058],
       [0.08312319],
       ...,
       [0.21551383],
       [0.12489984],
       [0.00385521]],

In [25]:
#Data load:
model = 'GBM'
X_train=np.loadtxt('X_train_%s'% model)
y_train=np.loadtxt('y_train_%s'% model)
y_train=y_train.reshape(len(y_train),1)
X_test=np.loadtxt('X_test_%s'% model)
y_test=np.loadtxt('y_test_%s'% model)
y_test=y_test.reshape(len(y_test),1)
X_val=np.loadtxt('X_val_%s'% model)
y_val=np.loadtxt('y_val_%s'% model)
y_val=y_val.reshape(len(y_val),1)
X_data_H=np.loadtxt('X_data_H_%s'% model)
y_data_H=np.loadtxt('y_data_H_%s'% model)
y_data_H=y_data_H.reshape(len(y_data_H),1)

#3 layers:
buildNN(3, ['elu','elu','elu'], [120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test, y_test, X_val, y_val, X_data_H, y_data_H,1,'GBM')
buildNN(3, ['leaky','leaky','leaky'], [120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test, y_test, X_val, y_val, X_data_H, y_data_H,2,'GBM')
buildNN(3, ['relu','relu','relu'], [120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test, y_test, X_val, y_val, X_data_H, y_data_H,3,'GBM')

#5 layers:
buildNN(5, ['elu','elu','elu','elu','elu'], [120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test, y_test, X_val, y_val, X_data_H, y_data_H,1,'GBM')
buildNN(5, ['leaky','leaky','leaky','leaky','leaky'], [120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test, y_test, X_val, y_val, X_data_H, y_data_H,2,'GBM')
buildNN(5, ['relu','relu','relu','relu','relu'], [120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test, y_test, X_val, y_val, X_data_H, y_data_H,3,'GBM')

#6 layers:
buildNN(6, ['elu','elu','elu','elu','elu','elu'], [120,120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test, y_test, X_val, y_val, X_data_H, y_data_H,1,'GBM')
buildNN(6, ['leaky','leaky','leaky','leaky','leaky','leaky'], [120,120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test, y_test, X_val, y_val, X_data_H, y_data_H,2,'GBM')
buildNN(6, ['relu','relu','relu','relu','relu','relu'], [120,120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test, y_test, X_val, y_val, X_data_H, y_data_H,3,'GBM')

training_size = 40000
X_train=np.loadtxt('X_train_{}_{}'.format(training_size,model))
y_train=np.loadtxt('y_train_{}_{}'.format(training_size,model))
y_train=y_train.reshape(len(y_train),1)     
X_val=np.loadtxt('X_val_{}_{}'.format(training_size,model))
y_val=np.loadtxt('y_val_{}_{}'.format(training_size,model))
y_val=y_val.reshape(len(y_val),1)
X_data_H=np.loadtxt('X_data_H_{}_{}'.format(training_size,model))
y_data_H=np.loadtxt('y_data_H_{}_{}'.format(training_size,model))
y_data_H=y_data_H.reshape(len(y_data_H),1)

buildNN_small_size(40000,3, ['elu','elu','elu'], [120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,1,'GBM')
buildNN_small_size(40000,3, ['leaky','leaky','leaky'], [120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,2,'GBM')
buildNN_small_size(40000,3, ['relu','relu','relu'], [120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,3,'GBM')

buildNN_small_size(40000,5, ['elu','elu','elu','elu','elu'], [120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,1,'GBM')
buildNN_small_size(40000,5, ['leaky','leaky','leaky','leaky','leaky'], [120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,2,'GBM')
buildNN_small_size(40000,5, ['relu','relu','relu','relu','relu'], [120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,3,'GBM')

buildNN_small_size(40000,6, ['elu','elu','elu','elu','elu','elu'], [120,120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,1,'GBM')
buildNN_small_size(40000,6, ['leaky','leaky','leaky','leaky','leaky','leaky'], [120,120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,2,'GBM')
buildNN_small_size(40000,6, ['relu','relu','relu','relu','relu','relu'], [120,120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,3,'GBM')

training_size = 80000
X_train=np.loadtxt('X_train_{}_{}'.format(training_size,model))
y_train=np.loadtxt('y_train_{}_{}'.format(training_size,model))
y_train=y_train.reshape(len(y_train),1)     
X_val=np.loadtxt('X_val_{}_{}'.format(training_size,model))
y_val=np.loadtxt('y_val_{}_{}'.format(training_size,model))
y_val=y_val.reshape(len(y_val),1)
X_data_H=np.loadtxt('X_data_H_{}_{}'.format(training_size,model))
y_data_H=np.loadtxt('y_data_H_{}_{}'.format(training_size,model))
y_data_H=y_data_H.reshape(len(y_data_H),1)

buildNN_small_size(80000,3, ['elu','elu','elu'], [120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,1,'GBM')
buildNN_small_size(80000,3, ['leaky','leaky','leaky'], [120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,2,'GBM')
buildNN_small_size(80000,3, ['relu','relu','relu'], [120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,3,'GBM')

buildNN_small_size(80000,5, ['elu','elu','elu','elu','elu'], [120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,1,'GBM')
buildNN_small_size(80000,5, ['leaky','leaky','leaky','leaky','leaky'], [120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,2,'GBM')
buildNN_small_size(80000,5, ['relu','relu','relu','relu','relu'], [120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,3,'GBM')

buildNN_small_size(80000,6, ['elu','elu','elu','elu','elu','elu'], [120,120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,1,'GBM')
buildNN_small_size(80000,6, ['leaky','leaky','leaky','leaky','leaky','leaky'], [120,120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,2,'GBM')
buildNN_small_size(80000,6, ['relu','relu','relu','relu','relu','relu'], [120,120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,3,'GBM')

training_size = 160000
X_train=np.loadtxt('X_train_{}_{}'.format(training_size,model))
y_train=np.loadtxt('y_train_{}_{}'.format(training_size,model))
y_train=y_train.reshape(len(y_train),1)     
X_val=np.loadtxt('X_val_{}_{}'.format(training_size,model))
y_val=np.loadtxt('y_val_{}_{}'.format(training_size,model))
y_val=y_val.reshape(len(y_val),1)
X_data_H=np.loadtxt('X_data_H_{}_{}'.format(training_size,model))
y_data_H=np.loadtxt('y_data_H_{}_{}'.format(training_size,model))
y_data_H=y_data_H.reshape(len(y_data_H),1)

buildNN_small_size(160000,3, ['elu','elu','elu'], [120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,1,'GBM')
buildNN_small_size(160000,3, ['leaky','leaky','leaky'], [120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,2,'GBM')
buildNN_small_size(160000,3, ['relu','relu','relu'], [120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,3,'GBM')

buildNN_small_size(160000,5, ['elu','elu','elu','elu','elu'], [120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,1,'GBM')
buildNN_small_size(160000,5, ['leaky','leaky','leaky','leaky','leaky'], [120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,2,'GBM')
buildNN_small_size(160000,5, ['relu','relu','relu','relu','relu'], [120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,3,'GBM')

buildNN_small_size(160000,6, ['elu','elu','elu','elu','elu','elu'], [120,120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,1,'GBM')
buildNN_small_size(160000,6, ['leaky','leaky','leaky','leaky','leaky','leaky'], [120,120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,2,'GBM')
buildNN_small_size(160000,6, ['relu','relu','relu','relu','relu','relu'], [120,120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,3,'GBM')

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00834951 total validation loss: 0.00174200
Epoch: 2 average loss: 0.00187342 total validation loss: 0.00114851
Epoch: 3 average loss: 0.00148383 total validation loss: 0.00093773
Epoch: 4 average loss: 0.00081317 total validation loss: 0.00024817
Epoch: 5 average loss: 0.00047215 total validation loss: 0.00014253
Epoch: 6 average loss: 0.00038232 total validation loss: 0.00014363
Epoch: 7 average loss: 0.00035932 total validation loss: 0.00016569
Epoch: 8 average loss: 0.00034298 total validation loss: 0.00018727
Epoch: 9 average loss: 0.00033704 total validation loss: 0.00009707
Epoch:10 average loss: 0.00032632 total validation loss: 0.00016517
Training Time : 0.9972876826922099 minutes 
total test loss of ['elu', 'elu', 'elu']: [0.00016065795]
total train loss of ['elu', 'elu', 'elu']: [0.0001622945]
[array([[0.34306604],
       [0.481048  ],
       [0.47806904],
       ...,
       [0.

Epoch: 4 average loss: 0.00050701 total validation loss: 0.00009072
Epoch: 5 average loss: 0.00040357 total validation loss: 0.00031767
Epoch: 6 average loss: 0.00034740 total validation loss: 0.00006860
Epoch: 7 average loss: 0.00030833 total validation loss: 0.00011675
Epoch: 8 average loss: 0.00029991 total validation loss: 0.00004522
Epoch: 9 average loss: 0.00028481 total validation loss: 0.00003663
Epoch:10 average loss: 0.00027769 total validation loss: 0.00018024
Training Time : 1.6952401320139567 minutes 
total test loss of ['leaky', 'leaky', 'leaky', 'leaky', 'leaky', 'leaky']: [0.00017793778]
total train loss of ['leaky', 'leaky', 'leaky', 'leaky', 'leaky', 'leaky']: [0.00017805782]
[array([[0.35822204],
       [0.4803683 ],
       [0.47521168],
       ...,
       [0.3877947 ],
       [0.11550868],
       [0.09107913]], dtype=float32)]
Average prediction time: 12.966763973236084
Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 

Epoch: 7 average loss: 0.00134784 total validation loss: 0.00015227
Epoch: 8 average loss: 0.00125313 total validation loss: 0.00017120
Epoch: 9 average loss: 0.00120179 total validation loss: 0.00013297
Epoch:10 average loss: 0.00115132 total validation loss: 0.00016105
Training Time : 0.3031052152315776 minutes 
total test loss of ['relu', 'relu', 'relu', 'relu', 'relu']: [0.00017294513]
total train loss of ['relu', 'relu', 'relu', 'relu', 'relu']: [0.00016943325]
[array([[0.33857337],
       [0.47468886],
       [0.4850966 ],
       ...,
       [0.3747962 ],
       [0.11349989],
       [0.10052343]], dtype=float32)]
Average prediction time: 9.124485651652018
Train on 36000 samples, validate on 4000 samples, test on 60000 samples
Epoch: 1 average loss: 0.05618049 total validation loss: 0.00423558
Epoch: 2 average loss: 0.00389065 total validation loss: 0.00214177
Epoch: 3 average loss: 0.00312067 total validation loss: 0.00167119
Epoch: 4 average loss: 0.00253334 total validation los

Epoch:10 average loss: 0.00090506 total validation loss: 0.00030189
Training Time : 0.5435671329498291 minutes 
total test loss of ['elu', 'elu', 'elu', 'elu', 'elu']: [0.00029340244]
total train loss of ['elu', 'elu', 'elu', 'elu', 'elu']: [0.00029666454]
[array([[0.33451515],
       [0.46341592],
       [0.46877584],
       ...,
       [0.35839784],
       [0.12598693],
       [0.09488524]], dtype=float32)]
Average prediction time: 10.446250438690186
Train on 72000 samples, validate on 8000 samples, test on 60000 samples
Epoch: 1 average loss: 0.02180430 total validation loss: 0.00025441
Epoch: 2 average loss: 0.00102474 total validation loss: 0.00022436
Epoch: 3 average loss: 0.00081462 total validation loss: 0.00014510
Epoch: 4 average loss: 0.00120025 total validation loss: 0.00030946
Epoch: 5 average loss: 0.00074814 total validation loss: 0.00010136
Epoch: 6 average loss: 0.00064650 total validation loss: 0.00018822
Epoch: 7 average loss: 0.00055711 total validation loss: 0.0000

total train loss of ['leaky', 'leaky', 'leaky']: [4.4678858e-05]
[array([[0.34199223],
       [0.48222467],
       [0.48283738],
       ...,
       [0.3746257 ],
       [0.12877162],
       [0.09997139]], dtype=float32)]
Average prediction time: 7.660082976023356
Train on 144000 samples, validate on 16000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01450156 total validation loss: 0.00048176
Epoch: 2 average loss: 0.00113718 total validation loss: 0.00025231
Epoch: 3 average loss: 0.00098581 total validation loss: 0.00012757
Epoch: 4 average loss: 0.00084110 total validation loss: 0.00022034
Epoch: 5 average loss: 0.00067286 total validation loss: 0.00012764
Epoch: 6 average loss: 0.00059764 total validation loss: 0.00021106
Epoch: 7 average loss: 0.00058322 total validation loss: 0.00012750
Epoch: 8 average loss: 0.00056695 total validation loss: 0.00018262
Epoch: 9 average loss: 0.00056596 total validation loss: 0.00012158
Epoch:10 average loss: 0.00056030 total validation

1.0885356148084004

In [26]:
#Data load:
model = 'BarrierDownIn'
X_train=np.loadtxt('X_train_%s'% model)
y_train=np.loadtxt('y_train_%s'% model)
y_train=y_train.reshape(len(y_train),1)
X_test=np.loadtxt('X_test_%s'% model)
y_test=np.loadtxt('y_test_%s'% model)
y_test=y_test.reshape(len(y_test),1)
X_val=np.loadtxt('X_val_%s'% model)
y_val=np.loadtxt('y_val_%s'% model)
y_val=y_val.reshape(len(y_val),1)
X_data_H=np.loadtxt('X_data_H_%s'% model)
y_data_H=np.loadtxt('y_data_H_%s'% model)
y_data_H=y_data_H.reshape(len(y_data_H),1)

#3 layers:
buildNN(3, ['elu','elu','elu'], [120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test, y_test, X_val, y_val, X_data_H, y_data_H,1,'BarrierDownIn')
buildNN(3, ['leaky','leaky','leaky'], [120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test, y_test, X_val, y_val, X_data_H, y_data_H,2,'BarrierDownIn')
buildNN(3, ['relu','relu','relu'], [120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test, y_test, X_val, y_val, X_data_H, y_data_H,3,'BarrierDownIn')

#5 layers:
buildNN(5, ['elu','elu','elu','elu','elu'], [120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test, y_test, X_val, y_val, X_data_H, y_data_H,1,'BarrierDownIn')
buildNN(5, ['leaky','leaky','leaky','leaky','leaky'], [120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test, y_test, X_val, y_val, X_data_H, y_data_H,2,'BarrierDownIn')
buildNN(5, ['relu','relu','relu','relu','relu'], [120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test, y_test, X_val, y_val, X_data_H, y_data_H,3,'BarrierDownIn')

#6 layers:
buildNN(6, ['elu','elu','elu','elu','elu','elu'], [120,120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test, y_test, X_val, y_val, X_data_H, y_data_H,1,'BarrierDownIn')
buildNN(6, ['leaky','leaky','leaky','leaky','leaky','leaky'], [120,120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test, y_test, X_val, y_val, X_data_H, y_data_H,2,'BarrierDownIn')
buildNN(6, ['relu','relu','relu','relu','relu','relu'], [120,120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test, y_test, X_val, y_val, X_data_H, y_data_H,3,'BarrierDownIn')

training_size = 40000
X_train=np.loadtxt('X_train_{}_{}'.format(training_size,model))
y_train=np.loadtxt('y_train_{}_{}'.format(training_size,model))
y_train=y_train.reshape(len(y_train),1)     
X_val=np.loadtxt('X_val_{}_{}'.format(training_size,model))
y_val=np.loadtxt('y_val_{}_{}'.format(training_size,model))
y_val=y_val.reshape(len(y_val),1)
X_data_H=np.loadtxt('X_data_H_{}_{}'.format(training_size,model))
y_data_H=np.loadtxt('y_data_H_{}_{}'.format(training_size,model))
y_data_H=y_data_H.reshape(len(y_data_H),1)

buildNN_small_size(40000,3, ['elu','elu','elu'], [120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,1,'BarrierDownIn')
buildNN_small_size(40000,3, ['leaky','leaky','leaky'], [120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,2,'BarrierDownIn')
buildNN_small_size(40000,3, ['relu','relu','relu'], [120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,3,'BarrierDownIn')

buildNN_small_size(40000,5, ['elu','elu','elu','elu','elu'], [120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,1,'BarrierDownIn')
buildNN_small_size(40000,5, ['leaky','leaky','leaky','leaky','leaky'], [120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,2,'BarrierDownIn')
buildNN_small_size(40000,5, ['relu','relu','relu','relu','relu'], [120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,3,'BarrierDownIn')

buildNN_small_size(40000,6, ['elu','elu','elu','elu','elu','elu'], [120,120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,1,'BarrierDownIn')
buildNN_small_size(40000,6, ['leaky','leaky','leaky','leaky','leaky','leaky'], [120,120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,2,'BarrierDownIn')
buildNN_small_size(40000,6, ['relu','relu','relu','relu','relu','relu'], [120,120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,3,'BarrierDownIn')

training_size = 80000
X_train=np.loadtxt('X_train_{}_{}'.format(training_size,model))
y_train=np.loadtxt('y_train_{}_{}'.format(training_size,model))
y_train=y_train.reshape(len(y_train),1)     
X_val=np.loadtxt('X_val_{}_{}'.format(training_size,model))
y_val=np.loadtxt('y_val_{}_{}'.format(training_size,model))
y_val=y_val.reshape(len(y_val),1)
X_data_H=np.loadtxt('X_data_H_{}_{}'.format(training_size,model))
y_data_H=np.loadtxt('y_data_H_{}_{}'.format(training_size,model))
y_data_H=y_data_H.reshape(len(y_data_H),1)

buildNN_small_size(80000,3, ['elu','elu','elu'], [120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,1,'BarrierDownIn')
buildNN_small_size(80000,3, ['leaky','leaky','leaky'], [120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,2,'BarrierDownIn')
buildNN_small_size(80000,3, ['relu','relu','relu'], [120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,3,'BarrierDownIn')

buildNN_small_size(80000,5, ['elu','elu','elu','elu','elu'], [120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,1,'BarrierDownIn')
buildNN_small_size(80000,5, ['leaky','leaky','leaky','leaky','leaky'], [120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,2,'BarrierDownIn')
buildNN_small_size(80000,5, ['relu','relu','relu','relu','relu'], [120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,3,'BarrierDownIn')

buildNN_small_size(80000,6, ['elu','elu','elu','elu','elu','elu'], [120,120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,1,'BarrierDownIn')
buildNN_small_size(80000,6, ['leaky','leaky','leaky','leaky','leaky','leaky'], [120,120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,2,'BarrierDownIn')
buildNN_small_size(80000,6, ['relu','relu','relu','relu','relu','relu'], [120,120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,3,'BarrierDownIn')

training_size = 160000
X_train=np.loadtxt('X_train_{}_{}'.format(training_size,model))
y_train=np.loadtxt('y_train_{}_{}'.format(training_size,model))
y_train=y_train.reshape(len(y_train),1)     
X_val=np.loadtxt('X_val_{}_{}'.format(training_size,model))
y_val=np.loadtxt('y_val_{}_{}'.format(training_size,model))
y_val=y_val.reshape(len(y_val),1)
X_data_H=np.loadtxt('X_data_H_{}_{}'.format(training_size,model))
y_data_H=np.loadtxt('y_data_H_{}_{}'.format(training_size,model))
y_data_H=y_data_H.reshape(len(y_data_H),1)

buildNN_small_size(160000,3, ['elu','elu','elu'], [120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,1,'BarrierDownIn')
buildNN_small_size(160000,3, ['leaky','leaky','leaky'], [120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,2,'BarrierDownIn')
buildNN_small_size(160000,3, ['relu','relu','relu'], [120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,3,'BarrierDownIn')

buildNN_small_size(160000,5, ['elu','elu','elu','elu','elu'], [120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,1,'BarrierDownIn')
buildNN_small_size(160000,5, ['leaky','leaky','leaky','leaky','leaky'], [120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,2,'BarrierDownIn')
buildNN_small_size(160000,5, ['relu','relu','relu','relu','relu'], [120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,3,'BarrierDownIn')

buildNN_small_size(160000,6, ['elu','elu','elu','elu','elu','elu'], [120,120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,1,'BarrierDownIn')
buildNN_small_size(160000,6, ['leaky','leaky','leaky','leaky','leaky','leaky'], [120,120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,2,'BarrierDownIn')
buildNN_small_size(160000,6, ['relu','relu','relu','relu','relu','relu'], [120,120,120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,3,'BarrierDownIn')

Train on 216000 samples, validate on 24000 samples, test on 60000 samples
Epoch: 1 average loss: 0.00907854 total validation loss: 0.00150846
Epoch: 2 average loss: 0.00144155 total validation loss: 0.00075361
Epoch: 3 average loss: 0.00107781 total validation loss: 0.00076149
Epoch: 4 average loss: 0.00096546 total validation loss: 0.00061067
Epoch: 5 average loss: 0.00048197 total validation loss: 0.00012388
Epoch: 6 average loss: 0.00028968 total validation loss: 0.00009294
Epoch: 7 average loss: 0.00023241 total validation loss: 0.00005947
Epoch: 8 average loss: 0.00021338 total validation loss: 0.00006027
Epoch: 9 average loss: 0.00020425 total validation loss: 0.00004651
Epoch:10 average loss: 0.00020191 total validation loss: 0.00007649
Training Time : 1.0574026664098104 minutes 
total test loss of ['elu', 'elu', 'elu']: [7.552904e-05]
total train loss of ['elu', 'elu', 'elu']: [7.392306e-05]
[array([[2.4549404e-01],
       [8.1776910e-02],
       [2.4796908e-01],
       ...,
  

Epoch: 2 average loss: 0.03415933 total validation loss: 0.03393989
Epoch: 3 average loss: 0.03414234 total validation loss: 0.03393989
Epoch: 4 average loss: 0.01420318 total validation loss: 0.00019774
Epoch: 5 average loss: 0.00066691 total validation loss: 0.00010484
Epoch: 6 average loss: 0.00037369 total validation loss: 0.00006389
Epoch: 7 average loss: 0.00032102 total validation loss: 0.00009356
Epoch: 8 average loss: 0.00025296 total validation loss: 0.00020148
Epoch: 9 average loss: 0.00024089 total validation loss: 0.00005630
Epoch:10 average loss: 0.00022481 total validation loss: 0.00020380
Training Time : 1.6661466161410015 minutes 
total test loss of ['leaky', 'leaky', 'leaky', 'leaky', 'leaky', 'leaky']: [0.00020341876]
total train loss of ['leaky', 'leaky', 'leaky', 'leaky', 'leaky', 'leaky']: [0.00020005017]
[array([[2.5429276e-01],
       [6.6287987e-02],
       [2.4361284e-01],
       ...,
       [2.9017302e-01],
       [2.9445622e-05],
       [2.1452518e-02]], dty

Epoch: 5 average loss: 0.00110565 total validation loss: 0.00014803
Epoch: 6 average loss: 0.00103918 total validation loss: 0.00021163
Epoch: 7 average loss: 0.00100325 total validation loss: 0.00013145
Epoch: 8 average loss: 0.00093307 total validation loss: 0.00018321
Epoch: 9 average loss: 0.00090107 total validation loss: 0.00010109
Epoch:10 average loss: 0.00088743 total validation loss: 0.00038564
Training Time : 0.3020374814669291 minutes 
total test loss of ['relu', 'relu', 'relu', 'relu', 'relu']: [0.00040429842]
total train loss of ['relu', 'relu', 'relu', 'relu', 'relu']: [0.00040117293]
[array([[2.2529134e-01],
       [6.1318047e-02],
       [2.2621404e-01],
       ...,
       [2.5204203e-01],
       [1.6223939e-04],
       [1.9869506e-02]], dtype=float32)]
Average prediction time: 9.877185026804606
Train on 36000 samples, validate on 4000 samples, test on 60000 samples
Epoch: 1 average loss: 0.06042028 total validation loss: 0.03345398
Epoch: 2 average loss: 0.03416083 to

Epoch: 7 average loss: 0.00123585 total validation loss: 0.00086177
Epoch: 8 average loss: 0.00114944 total validation loss: 0.00083777
Epoch: 9 average loss: 0.00109775 total validation loss: 0.00074401
Epoch:10 average loss: 0.00096279 total validation loss: 0.00023453
Training Time : 0.5724027514457702 minutes 
total test loss of ['elu', 'elu', 'elu', 'elu', 'elu']: [0.00024153656]
total train loss of ['elu', 'elu', 'elu', 'elu', 'elu']: [0.0002420576]
[array([[0.23513979],
       [0.08372167],
       [0.2457175 ],
       ...,
       [0.25915784],
       [0.00268564],
       [0.03499949]], dtype=float32)]
Average prediction time: 11.250432332356771
Train on 72000 samples, validate on 8000 samples, test on 60000 samples
Epoch: 1 average loss: 0.05087298 total validation loss: 0.03336408
Epoch: 2 average loss: 0.03443654 total validation loss: 0.03336408
Epoch: 3 average loss: 0.03441045 total validation loss: 0.03336408
Epoch: 4 average loss: 0.03449332 total validation loss: 0.03336

Epoch: 9 average loss: 0.00019019 total validation loss: 0.00006257
Epoch:10 average loss: 0.00017830 total validation loss: 0.00003574
Training Time : 0.7378769675890605 minutes 
total test loss of ['leaky', 'leaky', 'leaky']: [3.841363e-05]
total train loss of ['leaky', 'leaky', 'leaky']: [3.718482e-05]
[array([[0.23936109],
       [0.06969869],
       [0.23301812],
       ...,
       [0.2701664 ],
       [0.00041315],
       [0.02889568]], dtype=float32)]
Average prediction time: 5.965562661488851
Train on 144000 samples, validate on 16000 samples, test on 60000 samples
Epoch: 1 average loss: 0.01619736 total validation loss: 0.00025376
Epoch: 2 average loss: 0.00091864 total validation loss: 0.00019657
Epoch: 3 average loss: 0.00075265 total validation loss: 0.00014341
Epoch: 4 average loss: 0.00067356 total validation loss: 0.00008185
Epoch: 5 average loss: 0.00057496 total validation loss: 0.00008193
Epoch: 6 average loss: 0.00048253 total validation loss: 0.00008512
Epoch: 7 ave

total test loss of ['relu', 'relu', 'relu', 'relu', 'relu', 'relu']: [0.00011204518]
total train loss of ['relu', 'relu', 'relu', 'relu', 'relu', 'relu']: [0.0001081311]
[array([[2.4170326e-01],
       [8.6224668e-02],
       [2.4286655e-01],
       ...,
       [2.6991579e-01],
       [1.3147101e-05],
       [2.7457455e-02]], dtype=float32)]
Average prediction time: 11.677447954813639


1.023125962416331

In [31]:
#80000 -- 2 layers & 120 neurons & 'VG' :
set_of_layers,training_time = training_small_size(2,120,'VG',0.001,10,64,1,0.25,40000)

**********************Training 1 - ('elu', 'elu'):
Train on 35090 samples, validate on 3913 samples, test on 3913 samples
Epoch: 1 average loss: 0.04250360 total validation loss: 0.00851357
Epoch: 2 average loss: 0.00528968 total validation loss: 0.00208861
Epoch: 3 average loss: 0.00224299 total validation loss: 0.00167104
Epoch: 4 average loss: 0.00210824 total validation loss: 0.00171973
Epoch: 5 average loss: 0.00213097 total validation loss: 0.00171774
Epoch: 6 average loss: 0.00204291 total validation loss: 0.00161300
Epoch: 7 average loss: 0.00203793 total validation loss: 0.00177495
Epoch: 8 average loss: 0.00201172 total validation loss: 0.00162097
Epoch: 9 average loss: 0.00195552 total validation loss: 0.00175978
Epoch:10 average loss: 0.00196800 total validation loss: 0.00158265
Training Time : 0.13361939986546834 minutes 
total test loss of ('elu', 'elu'): [0.0015826533]
total train loss of ('elu', 'elu'): [0.0016000438]
[array([[0.04289333],
       [0.16723329],
       [0

Epoch: 1 average loss: 0.04812732 total validation loss: 0.00947409
Epoch: 2 average loss: 0.00729902 total validation loss: 0.00383209
Epoch: 3 average loss: 0.00279879 total validation loss: 0.00145754
Epoch: 4 average loss: 0.00164506 total validation loss: 0.00094961
Epoch: 5 average loss: 0.00127258 total validation loss: 0.00074082
Epoch: 6 average loss: 0.00110818 total validation loss: 0.00063196
Epoch: 7 average loss: 0.00102670 total validation loss: 0.00059281
Epoch: 8 average loss: 0.00098493 total validation loss: 0.00052464
Epoch: 9 average loss: 0.00097411 total validation loss: 0.00053174
Epoch:10 average loss: 0.00094516 total validation loss: 0.00051650
Training Time : 0.18608442147572835 minutes 
total test loss of ('relu', 'leaky'): [0.0005165017]
total train loss of ('relu', 'leaky'): [0.000511795]
[array([[0.01525595],
       [0.1847982 ],
       [0.0510929 ],
       ...,
       [0.07718524],
       [0.27354872],
       [0.09922545]], dtype=float32)]
Average predi

In [10]:
#Second run -- 5 layers & 120 neurons & 'GBM' :
set_of_layers,training_time = training(3,120,'VG')

**********************Training 1 - ('elu', 'elu', 'elu'):
Train on 211115 samples, validate on 23414 samples, test on 58682 samples
Epoch: 1 average loss: 0.00861444 total validation loss: 0.00160751
Epoch: 2 average loss: 0.00200281 total validation loss: 0.00130328
Epoch: 3 average loss: 0.00165278 total validation loss: 0.00122326
Epoch: 4 average loss: 0.00148026 total validation loss: 0.00102770
Epoch: 5 average loss: 0.00113528 total validation loss: 0.00048505
Epoch: 6 average loss: 0.00068733 total validation loss: 0.00042669
Epoch: 7 average loss: 0.00054952 total validation loss: 0.00029759
Epoch: 8 average loss: 0.00049951 total validation loss: 0.00025959
Epoch: 9 average loss: 0.00047030 total validation loss: 0.00021511
Epoch:10 average loss: 0.00043923 total validation loss: 0.00024189
Training Time : 0.917380952835083 minutes 
total test loss of ('elu', 'elu', 'elu'): [0.00024056603]
total train loss of ('elu', 'elu', 'elu'): [0.00024066857]
[array([[0.24348179],
      

Train on 211115 samples, validate on 23414 samples, test on 58682 samples
Epoch: 1 average loss: 0.00972984 total validation loss: 0.00113130
Epoch: 2 average loss: 0.00110708 total validation loss: 0.00048344
Epoch: 3 average loss: 0.00086994 total validation loss: 0.00033062
Epoch: 4 average loss: 0.00075508 total validation loss: 0.00029990
Epoch: 5 average loss: 0.00069610 total validation loss: 0.00029676
Epoch: 6 average loss: 0.00063215 total validation loss: 0.00027336
Epoch: 7 average loss: 0.00058639 total validation loss: 0.00026972
Epoch: 8 average loss: 0.00055560 total validation loss: 0.00027343
Epoch: 9 average loss: 0.00053977 total validation loss: 0.00025438
Epoch:10 average loss: 0.00051459 total validation loss: 0.00023804
Training Time : 0.9022403677304586 minutes 
total test loss of ('elu', 'relu', 'leaky'): [0.00023671414]
total train loss of ('elu', 'relu', 'leaky'): [0.00023768787]
[array([[0.26376534],
       [0.09931821],
       [0.23711695],
       ...,
   

Train on 211115 samples, validate on 23414 samples, test on 58682 samples
Epoch: 1 average loss: 0.00963847 total validation loss: 0.00029038
Epoch: 2 average loss: 0.00078534 total validation loss: 0.00014781
Epoch: 3 average loss: 0.00059140 total validation loss: 0.00010278
Epoch: 4 average loss: 0.00040633 total validation loss: 0.00013006
Epoch: 5 average loss: 0.00032734 total validation loss: 0.00011388
Epoch: 6 average loss: 0.00030218 total validation loss: 0.00009102
Epoch: 7 average loss: 0.00028550 total validation loss: 0.00005964
Epoch: 8 average loss: 0.00027684 total validation loss: 0.00008058
Epoch: 9 average loss: 0.00027182 total validation loss: 0.00008305
Epoch:10 average loss: 0.00026988 total validation loss: 0.00023957
Training Time : 0.9125250816345215 minutes 
total test loss of ('leaky', 'leaky', 'relu'): [0.00023597237]
total train loss of ('leaky', 'leaky', 'relu'): [0.00023786297]
[array([[0.24042225],
       [0.08834539],
       [0.2509396 ],
       ...,

Train on 211115 samples, validate on 23414 samples, test on 58682 samples
Epoch: 1 average loss: 0.00820383 total validation loss: 0.00051281
Epoch: 2 average loss: 0.00095761 total validation loss: 0.00048543
Epoch: 3 average loss: 0.00081617 total validation loss: 0.00043647
Epoch: 4 average loss: 0.00062503 total validation loss: 0.00024509
Epoch: 5 average loss: 0.00052949 total validation loss: 0.00026441
Epoch: 6 average loss: 0.00047956 total validation loss: 0.00019602
Epoch: 7 average loss: 0.00045345 total validation loss: 0.00020955
Epoch: 8 average loss: 0.00043127 total validation loss: 0.00026982
Epoch: 9 average loss: 0.00041526 total validation loss: 0.00017671
Epoch:10 average loss: 0.00038446 total validation loss: 0.00019661
Training Time : 0.897994315624237 minutes 
total test loss of ('relu', 'leaky', 'elu'): [0.000192774]
total train loss of ('relu', 'leaky', 'elu'): [0.00019325546]
[array([[0.24849844],
       [0.0964376 ],
       [0.24622433],
       ...,
      

In [16]:
set_of_layers,training_time = training(4,120,'VG')

**********************Training 1 - ['leaky', 'elu', 'elu', 'elu']:
Train on 211115 samples, validate on 23414 samples, test on 58682 samples
Epoch: 1 average loss: 0.00812701 total validation loss: 0.00037386
Epoch: 2 average loss: 0.00084184 total validation loss: 0.00031562
Epoch: 3 average loss: 0.00075245 total validation loss: 0.00041088
Epoch: 4 average loss: 0.00066411 total validation loss: 0.00024542
Epoch: 5 average loss: 0.00057869 total validation loss: 0.00023392
Epoch: 6 average loss: 0.00051591 total validation loss: 0.00024711
Epoch: 7 average loss: 0.00042518 total validation loss: 0.00021032
Epoch: 8 average loss: 0.00037239 total validation loss: 0.00011931
Epoch: 9 average loss: 0.00035366 total validation loss: 0.00038611
Epoch:10 average loss: 0.00034140 total validation loss: 0.00010273
Training Time : 1.1537724693616231 minutes 
total test loss of ['leaky', 'elu', 'elu', 'elu']: [9.944268e-05]
total train loss of ['leaky', 'elu', 'elu', 'elu']: [9.998691e-05]
[a

[array([[0.26589274],
       [0.09080753],
       [0.25539234],
       ...,
       [0.13779725],
       [0.12594321],
       [0.2766042 ]], dtype=float32)]
Average prediction time: 6.731467144461904
**********************Training 8 - ['leaky', 'elu', 'relu', 'leaky']:
Train on 211115 samples, validate on 23414 samples, test on 58682 samples
Epoch: 1 average loss: 0.00868495 total validation loss: 0.00048485
Epoch: 2 average loss: 0.00075711 total validation loss: 0.00022695
Epoch: 3 average loss: 0.00063149 total validation loss: 0.00016173
Epoch: 4 average loss: 0.00053596 total validation loss: 0.00016922
Epoch: 5 average loss: 0.00047545 total validation loss: 0.00024874
Epoch: 6 average loss: 0.00043881 total validation loss: 0.00012344
Epoch: 7 average loss: 0.00041271 total validation loss: 0.00013548
Epoch: 8 average loss: 0.00038234 total validation loss: 0.00009519
Epoch: 9 average loss: 0.00035655 total validation loss: 0.00009947
Epoch:10 average loss: 0.00034625 total valid

Epoch:10 average loss: 0.00024980 total validation loss: 0.00005860
Training Time : 1.1945666670799255 minutes 
total test loss of ['leaky', 'leaky', 'leaky', 'leaky']: [5.6325036e-05]
total train loss of ['leaky', 'leaky', 'leaky', 'leaky']: [5.729721e-05]
[array([[0.26237148],
       [0.09194331],
       [0.25844625],
       ...,
       [0.13736388],
       [0.12753469],
       [0.26987734]], dtype=float32)]
Average prediction time: 6.58629598039323
**********************Training 15 - ['leaky', 'leaky', 'leaky', 'relu']:
Train on 211115 samples, validate on 23414 samples, test on 58682 samples
Epoch: 1 average loss: 0.00907533 total validation loss: 0.00020054
Epoch: 2 average loss: 0.00086222 total validation loss: 0.00016129
Epoch: 3 average loss: 0.00062184 total validation loss: 0.00014055
Epoch: 4 average loss: 0.00043224 total validation loss: 0.00009789
Epoch: 5 average loss: 0.00034542 total validation loss: 0.00007962
Epoch: 6 average loss: 0.00032415 total validation loss: 

Epoch: 6 average loss: 0.00037807 total validation loss: 0.00008190
Epoch: 7 average loss: 0.00036513 total validation loss: 0.00006829
Epoch: 8 average loss: 0.00035220 total validation loss: 0.00014533
Epoch: 9 average loss: 0.00034323 total validation loss: 0.00015739
Epoch:10 average loss: 0.00032944 total validation loss: 0.00013163
Training Time : 1.1206585009892782 minutes 
total test loss of ['leaky', 'relu', 'elu', 'relu']: [0.000131747]
total train loss of ['leaky', 'relu', 'elu', 'relu']: [0.0001315818]
[array([[0.276467  ],
       [0.09627982],
       [0.27917776],
       ...,
       [0.14319956],
       [0.12821746],
       [0.2897923 ]], dtype=float32)]
Average prediction time: 5.725040249223054
**********************Training 22 - ['leaky', 'relu', 'leaky', 'elu']:
Train on 211115 samples, validate on 23414 samples, test on 58682 samples
Epoch: 1 average loss: 0.00788580 total validation loss: 0.00013546
Epoch: 2 average loss: 0.00068357 total validation loss: 0.00016690


In [21]:
model = 'VG'

X_test=np.loadtxt('X_test_{}'.format(model))
y_test=np.loadtxt('y_test_{}'.format(model))
y_test=y_test.reshape(len(y_test),1)

training_size = 40000
X_train=np.loadtxt('X_train_{}_{}'.format(training_size,model))
y_train=np.loadtxt('y_train_{}_{}'.format(training_size,model))
y_train=y_train.reshape(len(y_train),1)     
X_val=np.loadtxt('X_val_{}_{}'.format(training_size,model))
y_val=np.loadtxt('y_val_{}_{}'.format(training_size,model))
y_val=y_val.reshape(len(y_val),1)
X_data_H=np.loadtxt('X_data_H_{}_{}'.format(training_size,model))
y_data_H=np.loadtxt('y_data_H_{}_{}'.format(training_size,model))
y_data_H=y_data_H.reshape(len(y_data_H),1)
buildNN_small_size(40000,4, ['leaky','leaky','leaky','leaky'], [120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,2,'VG')



training_size = 80000
X_train=np.loadtxt('X_train_{}_{}'.format(training_size,model))
y_train=np.loadtxt('y_train_{}_{}'.format(training_size,model))
y_train=y_train.reshape(len(y_train),1)     
X_val=np.loadtxt('X_val_{}_{}'.format(training_size,model))
y_val=np.loadtxt('y_val_{}_{}'.format(training_size,model))
y_val=y_val.reshape(len(y_val),1)
X_data_H=np.loadtxt('X_data_H_{}_{}'.format(training_size,model))
y_data_H=np.loadtxt('y_data_H_{}_{}'.format(training_size,model))
y_data_H=y_data_H.reshape(len(y_data_H),1)
buildNN_small_size(80000,4, ['leaky','leaky','leaky','leaky'], [120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,2,'VG')


training_size = 160000
X_train=np.loadtxt('X_train_{}_{}'.format(training_size,model))
y_train=np.loadtxt('y_train_{}_{}'.format(training_size,model))
y_train=y_train.reshape(len(y_train),1)     
X_val=np.loadtxt('X_val_{}_{}'.format(training_size,model))
y_val=np.loadtxt('y_val_{}_{}'.format(training_size,model))
y_val=y_val.reshape(len(y_val),1)
X_data_H=np.loadtxt('X_data_H_{}_{}'.format(training_size,model))
y_data_H=np.loadtxt('y_data_H_{}_{}'.format(training_size,model))
y_data_H=y_data_H.reshape(len(y_data_H),1)
buildNN_small_size(160000,4, ['leaky','leaky','leaky','leaky'], [120,120,120,120], 0.001, 10, 64, 1, 0.25, X_train, y_train, X_test,y_test,X_val, y_val, X_data_H, y_data_H,2,'VG')

Train on 35090 samples, validate on 3913 samples, test on 58682 samples
Epoch: 1 average loss: 0.04966140 total validation loss: 0.00165003
Epoch: 2 average loss: 0.00164494 total validation loss: 0.00063378
Epoch: 3 average loss: 0.00103624 total validation loss: 0.00025062
Epoch: 4 average loss: 0.00091294 total validation loss: 0.00020800
Epoch: 5 average loss: 0.00080078 total validation loss: 0.00019869
Epoch: 6 average loss: 0.00075283 total validation loss: 0.00050131
Epoch: 7 average loss: 0.00081683 total validation loss: 0.00022471
Epoch: 8 average loss: 0.00067689 total validation loss: 0.00014091
Epoch: 9 average loss: 0.00089322 total validation loss: 0.00028021
Epoch:10 average loss: 0.00078572 total validation loss: 0.00018459
Training Time : 0.2465765635172526 minutes 
total test loss of ['leaky', 'leaky', 'leaky', 'leaky']: [0.00018174475]
total train loss of ['leaky', 'leaky', 'leaky', 'leaky']: [0.00018113537]
[array([[0.26488423],
       [0.10902275],
       [0.2441

0.8003827333450317